<a target="_blank" href="https://colab.research.google.com/github/Bo-Ni/ProteinMechanicsDiffusionDesign_pLDM/blob/main/notebook_for_colab/pLDM_inferring_standalong_colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## 0. Preparation:
### 0-1. Add packages

In [1]:
# !python --version

In [2]:
# !apt-get install python3.9

In [3]:
# import sys
# sys.path.pop(0)
# sys.path.insert(0, '/usr/bin/python3.9')

In [4]:
import os,sys
import math

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" #turn off CUDA if needed
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

path_1 = '/opt/bin/'
dssp_file = path_1+'mkdssp'

file_exists = os.path.exists(dssp_file)
if not (file_exists):
  print('\033[1;32m For the 1st run, ')
  # ==============================================
  print('a. Install omegafold...')
  # install omegafold
  # ref: https://github.com/HeliXonProtein/OmegaFold
  !pip install git+https://github.com/HeliXonProtein/OmegaFold.git

  # time-consuming step:
  # Downloading weights from https://helixon.s3.amazonaws.com/release1.pt to /root/.cache/omegafold_ckpt/model.pt
  !mkdir /root/.cache/omegafold_ckpt
  !wget https://helixon.s3.amazonaws.com/release1.pt -O /root/.cache/omegafold_ckpt/model.pt

  print('b. Install DSSP...')
  # download an mkdssp
  # ==============================================
  # download things
  print(os.popen(f"wget https://www.dropbox.com/s/v4azy9z9yojg1c6/mkdssp -P {path_1}").read())
  #
  !chmod u+x /opt/bin/mkdssp

else:
  print('This is not the first run... ')

 For the 1st run, 
a. Install omegafold...
  Cloning https://github.com/HeliXonProtein/OmegaFold.git to /tmp/pip-req-build-rtn3m6vg
  Running command git clone --filter=blob:none --quiet https://github.com/HeliXonProtein/OmegaFold.git /tmp/pip-req-build-rtn3m6vg
  Resolved https://github.com/HeliXonProtein/OmegaFold.git to commit 313c873ad190b64506a497c926649e15fcd88fcd
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.4 MB/s eta 0:00:00
  Created wheel for OmegaFold: filename=OmegaFold-0.0.0-py3-none-any.whl size=55664 sha256=327e601ea86253a594846ea033a4e4c3acfe3208535b460f4e4a09f253c1398d
  Stored in directory: /tmp/pip-ephem-wheel-cache-n70_867g/wheels/fe/57/01/3ce12996dd37debe3ee6a02e8748fffdd4df7b885c5bb8071d
Successfully built OmegaFold
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
 

In [5]:
# add some packages
try:
  print('\033[1;32m a. on Biopython...')
  from Bio.PDB import PDBParser
except ImportError as e:
  print(os.popen('pip install biopython').read())

try:
  print('\033[1;32m b. on kornia...')
  import kornia.augmentation
except ImportError as e:
  print(os.popen('pip install kornia').read())

try:
  print('\033[1;32m c. on einops...')
  from einops import rearrange, repeat, reduce
except ImportError as e:
  print(os.popen('pip install einops').read())

try:
  from einops_exts import rearrange_many, repeat_many, check_shape
except ImportError as e:
  print(os.popen('pip install einops-exts').read())

try:
  import pytorch_warmup as warmup
except ImportError as e:
  print(os.popen('pip install pytorch-warmup').read())

try:
  from ema_pytorch import EMA
except ImportError as e:
  print(os.popen('pip install ema-pytorch').read())

try:
  from accelerate import Accelerator, DistributedType, DistributedDataParallelKwargs
except ImportError as e:
  print(os.popen('pip install accelerate').read())

try:
  import py3Dmol
except ImportError as e:
  print(os.popen('pip install py3Dmol').read())

# added
try:
  import esm
except ImportError as e:
  print(os.popen('pip install fair-esm').read())

try:
  import torchinfo
except ImportError as e:
  print(os.popen('pip install torchinfo').read())

 a. on Biopython...
 b. on kornia...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 9.1 MB/s eta 0:00:00

 c. on einops...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.0 MB/s eta 0:00:00




### 0.2. copy src from github

In [6]:
import json, time, os, sys, glob

# # just script, no install is used
# if not os.path.isdir("ProteinMPNN"):
#   os.system("git clone -q https://github.com/dauparas/ProteinMPNN.git")
# sys.path.append('/content/ProteinMPNN/')

# ===================================================================

if not os.path.isdir("ProteinMechanicsDiffusionDesign_pLDM"):
  os.system("git clone -q https://github.com/Bo-Ni/ProteinMechanicsDiffusionDesign_pLDM.git")
# sys.path.append('/content/ProteinMechanicsDiffusionDesign_pLDM/ProteinMechanicsDiffusionDesign/')
sys.path.append('/content/ProteinMechanicsDiffusionDesign_pLDM/')

In [7]:
# a slient test
# import ProteinMechanicsDiffusionDesign.UtilityPack as UtilityPack
# import ProteinMechanicsDiffusionDesign.DataSetPack as DataSetPack
# import ProteinMechanicsDiffusionDesign.ModelPack as ModelPack
# import ProteinMechanicsDiffusionDesign.TrainerPack as TrainerPack
# import ProteinMechanicsDiffusionDesign.PostMDPack as PostMDPack
#
import PD_pLMProbXDiff.UtilityPack as UtilityPack
import PD_pLMProbXDiff.DataSetPack as DataSetPack
import PD_pLMProbXDiff.ModelPack as ModelPack
import PD_pLMProbXDiff.TrainerPack as TrainerPack
import PD_pLMProbXDiff.PostMDPack as PostMDPack

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


identify the device independently cuda:0


### 0.3. Download the model files

In [83]:
# just script, no install is used
this_working_path = '/content/working_results/'

if not os.path.isdir(this_working_path):
  print('Creating working path...')
  print(os.popen('mkdir '+this_working_path).read())
  print('Done.')
  print('Downing files...')

this_file = this_working_path+'model_pack.pickle'
file_exists = os.path.exists(this_file)
if not (file_exists):
  # download things
  this_link='https://www.dropbox.com/scl/fi/i2sull7ftjwrrzeaxo8v1/model_pack.pickle?rlkey=7wy5zynrl6m8azufklq3fy8ql&dl=0'
  cmd_line = f"wget -O {this_file} {this_link}"
  print(os.popen(cmd_line).read())

#
this_file = this_working_path+'data_pack.pickle'
file_exists = os.path.exists(this_file)
if not (file_exists):
  # download things
  this_link='https://www.dropbox.com/scl/fi/z7sz0q2nsjn85kyh68p86/data_pack.pickle?rlkey=bwm9fgf29ze8o516r155zg4gl&dl=0'
  cmd_line = f"wget -O {this_file} {this_link}"
  print(os.popen(cmd_line).read())

# add for dataset part
this_temp_path = this_working_path+'0_dataprocess_MD/'
if not os.path.isdir(this_temp_path):
  print('Creating data path...')
  print(os.popen('mkdir '+this_temp_path).read())
  print('Done.')
  print('Downing files...')
# add file if needed


# add for model part
this_temp_path = this_working_path+'1_model_SS/'
if not os.path.isdir(this_temp_path):
  print('Creating model path...')
  print(os.popen('mkdir '+this_temp_path).read())
  print('Done.')
  print('Downing files...')
# add file if needed
#
this_file = this_working_path+'1_model_SS/'+'trainer_save-model_pLDM.pt'
file_exists = os.path.exists(this_file)
if not (file_exists):
  # download things
  this_link='https://www.dropbox.com/s/09wyap14yrnoeom/trainer_save-model_pLDM.pt?dl=0'
  cmd_line = f"wget -O {this_file} {this_link}"
  print(os.popen(cmd_line).read())

Creating model path...

Done.
Downing files...



In [84]:
print(cmd_line)

wget -O /content/working_results/1_model_SS/trainer_save-model_pLDM.pt https://www.dropbox.com/s/09wyap14yrnoeom/trainer_save-model_pLDM.pt?dl=0


## 1. Working part

### 1.0. Check the floor

In [9]:
import os, sys

In [10]:
print('Here is : \n', os.popen('pwd').read())
print('What we get in hardware: \n', os.popen('nvidia-smi').read())

Here is : 
 /content

What we get in hardware: 
 Sun Oct  1 04:21:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    43W / 400W |      3MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                       

In [11]:
import torch
print("What we have in software: \n Torch version:", torch.__version__)
print('Python: ', sys.version) # no switch case code

What we have in software: 
 Torch version: 1.12.0+cu113
Python:  3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [12]:
import torch
print("What we have in software: \n Torch version:", torch.__version__)
print('Python: ', sys.version) # no switch case code

What we have in software: 
 Torch version: 1.12.0+cu113
Python:  3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [13]:
print('What hardware the software see:')
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu"
)
print(device)
num_of_gpus = torch.cuda.device_count()
print("# of GPU", num_of_gpus)

What hardware the software see:
cuda:0
# of GPU 1


In [14]:
torch.cuda.empty_cache()

### 1.1. Setup the problem

In [38]:
# prepare for package debugging
# for debug
import importlib
import json
import matplotlib.pyplot as plt

In [39]:
# import PD_pLMProbXDiff.UtilityPack as UtilityPack
# # run this when updating the package
# importlib.reload(UtilityPack)

In [40]:
# ===============================================
# Global control key setup
# ===============================================
# Control keys:
# This one will be directly modified for each task
CKeys = dict(
    #
    Running_Type=2, # 1-local:engaging cluster, 2-supercould cluster, 3-google colab, 4-local ubuntu
    #
    # Working_Mode=1, # 1-training, 2-sampling for test
    Working_Mode=2,
    #
    # IF_FirstRun=1,  # 1-1st run of training; otherwise, # of training run
    IF_FirstRun=2,  # 1-1st run of training; otherwise, # of training run
    #
    # 1-SecStr-ModelB, 2-MD-ModelB, 3-SecStr-ModelA, 4-MD-ModelA
    # 5-SecStr-ModelB-Embdding, 6-MD-ModelB, 7-SecStr-ModelA-pLM, 8-MD-ModelA
    # 9-MD-Predictor-ModelB, 10-
    # 11-MD-ModelB
    Problem_ID=11, # 8, # 6,
    #
    # Debug=1, # 1-debug mode on; add more debug keys for different blocks
    Debug=0, # 1-debug mode on; add more debug keys for different blocks
    #
    Debug_DataSet=1,
    Debug_Model=1
    #
    # Debug=0
)

In [41]:

if CKeys['Debug']==1:
    # add some
    CKeys['SlientRun']=0 # 1-save figure into files; 0-show figures
    # detailed debug keys
    # 1. for model dimension
    CKeys['Debug_DataPack']=1
    CKeys['Debug_ModelPack']=1
    CKeys['Debug_TrainerPack']=1
    # 2. for trainer part
    CKeys['Debug_DataPack']=1
    CKeys['Debug_ModelPack']=0
    CKeys['Debug_TrainerPack']=3
    # #
    # CKeys['testratio']=0.15 # for small ForcPath problem
    CKeys['testratio']=0.10 # for large ForcPath problem

else:
    # for real working run
    CKeys['SlientRun']=1
    #
    CKeys['Debug_DataPack'] = 0
    CKeys['Debug_ModelPack'] = 0
    CKeys['Debug_TrainerPack'] = 0 # 2 # 1
    # add some for training
    CKeys['epochs'] = 4000-3250 # 1000 # 200
    CKeys['print_loss_every_this_epochs']=50 # 5
    CKeys['sample_every_this_epochs']=100 # 50 # 20
    CKeys['save_model_every_this_epochs']=50 # 20
    # #
    # # add some for training
    # CKeys['epochs'] = 2000 # 1000 # 200
    # CKeys['print_loss_every_this_epochs']=20 # 5
    # CKeys['sample_every_this_epochs']=50 # 50 # 20
    # CKeys['save_model_every_this_epochs']=50 # 20
    #
    # CKeys['testratio']=0.15 # for small ForcPath problem
    CKeys['testratio']=0.10 # for large ForcPath problem
    #
    # # add some for training
    # CKeys['epochs'] = 4 # 1000 # 200
    # CKeys['print_loss_every_this_epochs']=1 # 5
    # CKeys['sample_every_this_epochs']=1 # 50 # 20
    # CKeys['save_model_every_this_epochs']=2 # 20

# for check
print(json.dumps(CKeys, indent=4))


{
    "Running_Type": 2,
    "Working_Mode": 2,
    "IF_FirstRun": 2,
    "Problem_ID": 11,
    "Debug": 0,
    "Debug_DataSet": 1,
    "Debug_Model": 1,
    "SlientRun": 1,
    "Debug_DataPack": 0,
    "Debug_ModelPack": 0,
    "Debug_TrainerPack": 0,
    "epochs": 750,
    "print_loss_every_this_epochs": 50,
    "sample_every_this_epochs": 100,
    "save_model_every_this_epochs": 50,
    "testratio": 0.1
}


In [42]:

# Problem type:
print('Problem type: ', CKeys['Problem_ID'])
print('Debug mode: ', CKeys['Debug'])
print('Working mode: ', CKeys['Working_Mode'])


Problem type:  11
Debug mode:  0
Working mode:  2


In [43]:

# ===========================================================
# Parameter key:
# ===========================================================
#
PKeys = {}
# define this one according the running environment
# add more if needed
# if CKeys['Running_Type']==1 or CKeys['Running_Type']==4:
# if CKeys['Running_Type']==1:
#
#
root_path = '/home/gridsan/bni/Test_ground/jupyter/1_git_project/sort_pdb_database_0/Local_Store/'
if CKeys['Debug']==1:
    # PKeys['prefix']='../Local_Store/For_16_0/'
    # use the absolute path for transformability
    PKeys['prefix']=root_path+'For_20_0/'

if CKeys['Debug']!=1:
    # PKeys['prefix']='../Local_Store/For_16_1/'
    PKeys['prefix']=root_path+'For_20_1/'
    PKeys['prefix']='/home/gridsan/bni/16_WG_git_sort_pdb_database_0/11_pLMProb_Diff_SMD_ModelB_embed_640/0_Training/'
    # add one for google colab
    PKeys['prefix']=this_working_path # '/content/11_pLMProb_Diff_SMD_ModelB_embed_640/0_Training/'

# store the data pack after processing
PKeys['pk_data_pack']=PKeys['prefix']+'data_pack.pickle'
PKeys['pk_model_pack']=PKeys['prefix']+'model_pack.pickle'
# PKeys[]

In [44]:
# print(CKeys['Running_Type'])
# print(CKeys['Working_Mode']==1 and CKeys['IF_FirstRun']==1)
# print(PKeys['prefix'])

In [45]:

#
# clean EVERYTHING in the dir if 1st
#
if CKeys['Working_Mode']==1 and CKeys['IF_FirstRun']==1:
    if os.path.exists(PKeys['prefix']):
        cmd_line=f"rm -r {PKeys['prefix']}"
        print("clean the slade...")
        print(f"excute {cmd_line}")
        os.popen(cmd_line).read()
        #
    # create dir for working space
    UtilityPack.create_path(PKeys['prefix'])

In [46]:
# ========================================================================
# prepare the csv files
# ========================================================================

if CKeys['Problem_ID']==1 or CKeys['Problem_ID']==3 \
or CKeys['Problem_ID']==5 or CKeys['Problem_ID']==7:
#     SS_csv_file = PKeys['prefix']+'PROTEIN_Mar18_2022_SECSTR_ALL.csv'

#     file_exists = os.path.exists(SS_csv_file)
#     if not (file_exists):
#         print('Downing the csv file...')
#         print(os.popen(f"wget https://www.dropbox.com/s/7o7s15w9qr6z76y/PROTEIN_Mar18_2022_SECSTR_ALL.csv -P {PKeys['prefix']}").read())
#         print('Done.')
#     else:
#         print("Already there")
    # +
    # SS_csv_file = '/home/gridsan/bni/Test_ground/jupyter/1_git_project/sort_pdb_database_0/Local_Store/'+'PROTEIN_Mar18_2022_SECSTR_ALL.csv'
    SS_csv_file = root_path+'Local_Store/'+'PROTEIN_Mar18_2022_SECSTR_ALL.csv'

if CKeys['Problem_ID']==2 or CKeys['Problem_ID']==4 \
or CKeys['Problem_ID']==6 or CKeys['Problem_ID']==8 \
or CKeys['Problem_ID']==11:
    # to be copied locally
    # MD_smo_csv_file = '/home/gridsan/bni/Test_ground/jupyter/1_git_project/sort_pdb_database_0/Local_Store/For_1/ForTrain_recon_BSDB_LE_64_smd_disp_forc_df_smo.csv'
    # first debug with LE_64, to be update into LE_128
    MD_smo_csv_file = root_path+'For_1/ForTrain_recon_BSDB_LE_64_smd_disp_forc_df_smo.csv'
    MD_smo_pk_file  = root_path+'For_1/ForTrain_recon_BSDB_LE_64_smd_disp_forc_df_smo_shared.pk'
    # into LE_128
    MD_smo_pk_file  = root_path+'For_1/ForTrain_recon_BSDB_LE_128_smd_disp_forc_df_smo.pk'

### 1.2. Handle the dataset

In [47]:
import pickle
import numpy as np
import pandas as pd

In [48]:
# import PD_pLMProbXDiff.DataSetPack as DataSetPack
# importlib.reload(DataSetPack)

In [49]:
print('On Problem: ', CKeys['Problem_ID'])

On Problem:  11


In [50]:
print(this_working_path)

/content/working_results/


In [51]:
# on the sec_str csv file: SecStr
# try to convey all para via one key
# ====================================================
# add some new keys for dataset
# ====================================================
# for data washing: only for 1st training cycle
if CKeys['Working_Mode']==1 and CKeys['IF_FirstRun']==1:

    if CKeys['Problem_ID']==1:
        pass
        # print("1")
        # # +++++++++++++++++++++++++++++++++++++
        # # SecStr as input seq
        # DataKeys={}
        # DataKeys['data_dir']=PKeys['prefix']+'0_dataprocess_SS/'
        # # screening rules
        # DataKeys['min_AA_seq_len']=0
        # DataKeys['max_AA_seq_len']=128
        # # X and Y processing
        # DataKeys['Xnormfac']=9.
        # DataKeys['ynormfac']=21.
        # DataKeys['tokenizer_X']=None
        # DataKeys['tokenizer_y']=None
        # # + for AA embending using ESM
        # DataKeys['ESM-2_Model']='esm2_t33_650M_UR50D'
        # # deliver
        # DataKeys['batch_size']=256
        # DataKeys['batch_size']=200
        # DataKeys['testset_ratio']=0.1
        # DataKeys['maxdata']=99999999999999999
        # # add the folder for Data part
        # UtilityPack.create_path(DataKeys['data_dir'])

    elif CKeys['Problem_ID']==2:
        pass
        print("2")
#         # ++++++++++++++++++++++++++++++++++++++
#         # MD record as the input seq
#         #
#         # try to convey all para via one key
#         DataKeys={}
#         # ======================================
#         # keys for "screen_dataset_MD"
#         DataKeys['data_dir']=PKeys['prefix']+'0_dataprocess_MD/'
#         # add the folder
#         UtilityPack.create_path(DataKeys['data_dir'])

#         # screening rules
#         DataKeys['min_AA_seq_len']=0
#         DataKeys['max_AA_seq_len']=64
#         DataKeys['max_Force_cap']=1000
#         # special ones
#         # change text arr into np arr
#         DataKeys['arr_key']=[
#             'posi_data','pull_data','forc_data',
#             'gap_data','normalized_gap_data',
#             'pull_gap_data', 'normalized_pull_gap_data',
#             'sample_NormPullGap_data','sample_FORCEpN_data']

#         df_raw, protein_df = DataSetPack.screen_dataset_MD(
#             file_path=MD_smo_csv_file,
#             PKeys=DataKeys, # to be updated
#             CKeys=CKeys,
#         )

#         # save the dataframe
#         pd.to_pickle(protein_df, DataKeys['data_dir']+'protein_df.pk')
#         pd.to_pickle(df_raw, DataKeys['data_dir']+'df_raw.pk')

#         # ======================================
#         # keys for 2nd function
#         DataKeys['X_Key']='sample_FORCEpN_data' # or 'Max_Smo_Force'
#         #
#         DataKeys['tokenizer_X']=None # will not be used
#         DataKeys['tokenizer_y']=None # to be created
#         DataKeys['Xnormfac'] = np.max(protein_df['Max_Smo_Force'])
#         print('Normalization factor for force: ', DataKeys['Xnormfac'])
#         DataKeys['ynormfac']=21. # old force diffusion model 22.
#         #
#         DataKeys['batch_size']=256
#         DataKeys['testset_ratio']=0.15
#         DataKeys['maxdata']=99999999991000

    elif CKeys['Problem_ID']==3:
        pass
        # print("3")
        # # +++++++++++++++++++++++++++++++++++++
        # # SecStr as input seq
        # DataKeys={}
        # DataKeys['data_dir']=PKeys['prefix']+'0_dataprocess_SS_ModelA/'
        # # screening rules
        # DataKeys['min_AA_seq_len']=0
        # DataKeys['max_AA_seq_len']=64 # 128
        # DataKeys['max_text_len']=8
        # # X and Y processing
        # DataKeys['Xnormfac']=1.
        # DataKeys['ynormfac']=22. # 21.
        # DataKeys['tokenizer_X']=None
        # DataKeys['tokenizer_y']=None
        # # deliver
        # DataKeys['batch_size']=512
        # # for debug purpose
        # # DataKeys['batch_size']=1
        # DataKeys['testset_ratio']= 0.1
        # DataKeys['maxdata']=99999999999999999
        # # add the folder for Data part
        # UtilityPack.create_path(DataKeys['data_dir'])

    elif CKeys['Problem_ID']==4:
        pass

#         print("4: input text condition, output sequence...")
#         # ++++++++++++++++++++++++++++++++++++++
#         # MD record as the input seq
#         #
#         # try to convey all para via one key
#         DataKeys={}
#         # ======================================
#         # keys for "screen_dataset_MD"
#         DataKeys['data_dir']=PKeys['prefix']+'0_dataprocess_MD/'
#         # add the folder
#         UtilityPack.create_path(DataKeys['data_dir'])

#         # screening rules
#         DataKeys['min_AA_seq_len']=0
#         DataKeys['max_AA_seq_len']=64
#         DataKeys['max_text_len']=2
#         DataKeys['max_Force_cap']=1000
#         # special ones
#         # change text arr into np arr
#         DataKeys['arr_key']=[
#             'posi_data','pull_data','forc_data',
#             'gap_data','normalized_gap_data',
#             'pull_gap_data', 'normalized_pull_gap_data',
#             'sample_NormPullGap_data','sample_FORCEpN_data']

#         df_raw, protein_df = DataSetPack.screen_dataset_MD(
#             file_path=MD_smo_csv_file,
#             PKeys=DataKeys, # to be updated
#             CKeys=CKeys,
#         )

#         # save the dataframe
#         pd.to_pickle(protein_df, DataKeys['data_dir']+'protein_df.pk')
#         pd.to_pickle(df_raw, DataKeys['data_dir']+'df_raw.pk')

#         # ======================================
#         # keys for 2nd function
#         DataKeys['X_Key']=['Max_Smo_Force','Int_Smo_ForcPull'] # 'sample_FORCEpN_data' # or 'Max_Smo_Force'
#         #
#         DataKeys['tokenizer_X']=None # will not be used
#         DataKeys['tokenizer_y']=None # to be created
#         #
#         print('Normalization factor for force: ',
#               np.max(protein_df['Max_Smo_Force']))
#         print('Normalization factor for toughness: ',
#               np.max(protein_df['Int_Smo_ForcPull']))
#         #
#         DataKeys['Xnormfac'] = np.array([
#             np.max(protein_df['Max_Smo_Force']),
#             np.max(protein_df['Int_Smo_ForcPull'])
#         ])
#         #
#         DataKeys['ynormfac']=21. # old force diffusion model 22.
#         #
#         DataKeys['batch_size']=256
#         DataKeys['testset_ratio']=0.15
#         DataKeys['maxdata']=99999999991000

    # /////////////////////////////////////////////////////////////
    # try embedding
    # \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
    elif CKeys['Problem_ID']==5:
        pass

#         print("5")
#         # +++++++++++++++++++++++++++++++++++++
#         # SecStr as input seq
#         DataKeys={}
#         DataKeys['data_dir']=PKeys['prefix']+'0_dataprocess_SS/'
#         # screening rules
#         DataKeys['min_AA_seq_len']=0
#         DataKeys['max_AA_seq_len']=128
#         # X and Y processing
#         DataKeys['Xnormfac']=9.
#         DataKeys['ynormfac']=1 # for ESM # 21.
#         DataKeys['tokenizer_X']=None
#         DataKeys['tokenizer_y']=None
#         # + for AA embending using ESM
#         DataKeys['ESM-2_Model']='esm2_t33_650M_UR50D'
#         # add for embedding space
#         DataKeys['image_channels']=1280
#         # deliver
#         DataKeys['batch_size']=256
#         DataKeys['batch_size']=256 # 0 # 200
#         DataKeys['testset_ratio']=CKeys['testratio'] # 0.1
#         DataKeys['maxdata']=99999999999999999

#         # add the folder for Data part
#         UtilityPack.create_path(DataKeys['data_dir'])
    #
    elif CKeys['Problem_ID']==6:
        print("6, input MD ForcePath, output AA sequence")
        # ++++++++++++++++++++++++++++++++++++++
        # MD record as the tokenized input seq
        #
        # try to convey all para via one key
        DataKeys={}
        # ======================================
        # keys for "screen_dataset_MD"
        DataKeys['data_dir']=PKeys['prefix']+'0_dataprocess_MD/'
        # add the folder
        UtilityPack.create_path(DataKeys['data_dir'])

        # screening rules
        DataKeys['min_AA_seq_len']=0
        # DataKeys['max_AA_seq_len']=64
        DataKeys['max_AA_seq_len']=128
        DataKeys['max_Force_cap']=1000
#         # ---------------------------------------
#         # special ones
#         # change text arr into np arr
#         DataKeys['arr_key']=[
#             'posi_data','pull_data','forc_data',
#             'gap_data','normalized_gap_data',
#             'pull_gap_data', 'normalized_pull_gap_data',
#             'sample_NormPullGap_data','sample_FORCEpN_data']

#         df_raw, protein_df = DataSetPack.screen_dataset_MD(
#             csv_file=MD_smo_csv_file,
#             pk_file=None,
#             PKeys=DataKeys, # to be updated
#             CKeys=CKeys,
#         )
        # ++++++++++++++++++++++++++++++++++++++
        df_raw, protein_df = DataSetPack.screen_dataset_MD(
            csv_file=None,
            pk_file=MD_smo_pk_file,
            PKeys=DataKeys, # to be updated
            CKeys=CKeys,
        )

        # save the dataframe
        pd.to_pickle(protein_df, DataKeys['data_dir']+'protein_df.pk')
        pd.to_pickle(df_raw, DataKeys['data_dir']+'df_raw.pk')

        # ======================================
        # keys for 2nd function
        DataKeys['X_Key']='sample_FORCEpN_data' # or 'Max_Smo_Force'
        #
        DataKeys['tokenizer_X']=None # will not be used
        DataKeys['tokenizer_y']=None # to be created
        # think about this: update this one if necessary
        # DataKeys['Xnormfac'] = np.max(protein_df['Max_Smo_Force'])
        DataKeys['Xnormfac'] = 750.

        print('Normalization factor for force: ', DataKeys['Xnormfac'])
        DataKeys['ynormfac']=1. # not used as esm is used # 21. # old force diffusion model 22.
        #
        DataKeys['batch_size']=256
        DataKeys['testset_ratio']=0.15
        DataKeys['maxdata']=99999999991000
        # ++ for pLM
        # for AA embending using ESM
        DataKeys['ESM-2_Model']='esm2_t33_650M_UR50D'
        # add for embedding space
        DataKeys['image_channels']=1280
        #
        DataKeys['ESM-2_Model']='esm2_t12_35M_UR50D'
        DataKeys['image_channels']=480


    elif CKeys['Problem_ID']==7:
        pass
        # print("7")
        # # +++++++++++++++++++++++++++++++++++++
        # # SecStr text as input seq
        # DataKeys={}
        # DataKeys['data_dir']=PKeys['prefix']+'0_dataprocess_SS_ModelA/'
        # # screening rules
        # DataKeys['min_AA_seq_len']=0
        # DataKeys['max_AA_seq_len']=64 # 128
        # DataKeys['max_text_len']=8
        # # X and Y processing
        # DataKeys['Xnormfac']=1.
        # DataKeys['ynormfac']=1. # for ESM # 21. 22. # 21.
        # DataKeys['tokenizer_X']=None
        # DataKeys['tokenizer_y']=None
        # # deliver
        # DataKeys['batch_size']=512
        # # for debug purpose
        # # DataKeys['batch_size']=1
        # DataKeys['testset_ratio']= CKeys['testratio'] # 0.1
        # DataKeys['maxdata']=99999999999999999
        # # + for AA embending using ESM
        # DataKeys['ESM-2_Model']='esm2_t33_650M_UR50D'
        # # add for embedding space
        # DataKeys['image_channels']=1280
        # #
        # # add the folder for Data part
        # UtilityPack.create_path(DataKeys['data_dir'])

    elif CKeys['Problem_ID']==8:
        #
        print("8: input text condition, output sequence...")
        # ++++++++++++++++++++++++++++++++++++++
        # MD record as the input seq
        #
        # try to convey all para via one key
        DataKeys={}
        # ======================================
        # keys for "screen_dataset_MD"
        DataKeys['data_dir']=PKeys['prefix']+'0_dataprocess_MD/'
        # add the folder
        UtilityPack.create_path(DataKeys['data_dir'])

        # screening rules
        DataKeys['min_AA_seq_len']=0
        DataKeys['max_AA_seq_len']=64
        DataKeys['max_AA_seq_len']=128
        DataKeys['max_text_len']=2
        DataKeys['max_Force_cap']=1000
#         # ---------------------------------------------------------
#         # special ones
#         # change text arr into np arr
#         DataKeys['arr_key']=[
#             'posi_data','pull_data','forc_data',
#             'gap_data','normalized_gap_data',
#             'pull_gap_data', 'normalized_pull_gap_data',
#             'sample_NormPullGap_data','sample_FORCEpN_data']

#         df_raw, protein_df = DataSetPack.screen_dataset_MD(
#             # # --
#             # file_path=MD_smo_csv_file,
#             # ++
#             csv_file=MD_smo_csv_file,
#             pk_file=None,
#             PKeys=DataKeys, # to be updated
#             CKeys=CKeys,
#         )
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        df_raw, protein_df = DataSetPack.screen_dataset_MD(
            csv_file=None,
            pk_file=MD_smo_pk_file,
            PKeys=DataKeys, # to be updated
            CKeys=CKeys,
        )

        # save the dataframe
        pd.to_pickle(protein_df, DataKeys['data_dir']+'protein_df.pk')
        pd.to_pickle(df_raw, DataKeys['data_dir']+'df_raw.pk')

        # ======================================
        # keys for 2nd function
        DataKeys['X_Key']=['Max_Smo_Force','Int_Smo_ForcPull'] # 'sample_FORCEpN_data' # or 'Max_Smo_Force'
        #
        DataKeys['tokenizer_X']=None # will not be used
        DataKeys['tokenizer_y']=None # will not be used # to be created
        #
        print('Normalization factor for force: ',
              np.max(protein_df['Max_Smo_Force']))
        print('Normalization factor for toughness: ',
              np.max(protein_df['Int_Smo_ForcPull']))
        #
        DataKeys['Xnormfac'] = np.array([
            np.max(protein_df['Max_Smo_Force']),
            np.max(protein_df['Int_Smo_ForcPull'])
        ])
        #
        DataKeys['ynormfac']=1.0 # not used in esm # 21. # old force diffusion model 22.
        #
        DataKeys['batch_size']=256
        DataKeys['testset_ratio']=CKeys['testratio'] # 0.15
        DataKeys['maxdata']=99999999991000
        # + for AA embending using ESM
        DataKeys['ESM-2_Model']='esm2_t33_650M_UR50D'
        # add for embedding space
        DataKeys['image_channels']=1280

    elif CKeys['Problem_ID']==11:
        # copied from Problem_ID=6
        #
        print("11, input MD ForcePath, output AA sequence")
        # ++++++++++++++++++++++++++++++++++++++
        # MD record as the tokenized input seq
        #
        # try to convey all para via one key
        DataKeys={}
        # ======================================
        # keys for "screen_dataset_MD"
        DataKeys['data_dir']=PKeys['prefix']+'0_dataprocess_MD/'
        # add the folder
        UtilityPack.create_path(DataKeys['data_dir'])

        # screening rules
        DataKeys['min_AA_seq_len']=0
        # DataKeys['max_AA_seq_len']=64
        DataKeys['max_AA_seq_len']=128
        DataKeys['max_Force_cap']=1000
#         # ---------------------------------------
#         # special ones
#         # change text arr into np arr
#         DataKeys['arr_key']=[
#             'posi_data','pull_data','forc_data',
#             'gap_data','normalized_gap_data',
#             'pull_gap_data', 'normalized_pull_gap_data',
#             'sample_NormPullGap_data','sample_FORCEpN_data']

#         df_raw, protein_df = DataSetPack.screen_dataset_MD(
#             csv_file=MD_smo_csv_file,
#             pk_file=None,
#             PKeys=DataKeys, # to be updated
#             CKeys=CKeys,
#         )
        # ++++++++++++++++++++++++++++++++++++++
        df_raw, protein_df = DataSetPack.screen_dataset_MD(
            csv_file=None,
            pk_file=MD_smo_pk_file,
            PKeys=DataKeys, # to be updated
            CKeys=CKeys,
        )

        # save the dataframe
        pd.to_pickle(protein_df, DataKeys['data_dir']+'protein_df.pk')
        pd.to_pickle(df_raw, DataKeys['data_dir']+'df_raw.pk')

        # ======================================
        # keys for 2nd function
        DataKeys['X_Key']='sample_FORCEpN_data' # or 'Max_Smo_Force'
        #
        DataKeys['tokenizer_X']=None # will not be used
        DataKeys['tokenizer_y']=None # to be created
        # think about this: update this one if necessary
        # DataKeys['Xnormfac'] = np.max(protein_df['Max_Smo_Force'])
        DataKeys['Xnormfac'] = 750.

        print('Normalization factor for force: ', DataKeys['Xnormfac'])
        DataKeys['ynormfac']=1. # not used as esm is used # 21. # old force diffusion model 22.
        #
        DataKeys['batch_size']=256
        DataKeys['testset_ratio']=0.15
        DataKeys['maxdata']=99999999991000
        # ++ for pLM
        # for AA embending using ESM
        DataKeys['ESM-2_Model']='esm2_t33_650M_UR50D'
        # add for embedding space
        # DataKeys['image_channels']=1280
        #
        # DataKeys['ESM-2_Model']='esm2_t12_35M_UR50D'
        # # DataKeys['image_channels']=480
        #
        # DataKeys['ESM-2_Model']='esm2_t36_3B_UR50D'
        # DataKeys['image_channels']=2560
        #
        DataKeys['ESM-2_Model']='esm2_t30_150M_UR50D'
        # DataKeys['image_channels']=640

        # only use the probability part
        DataKeys['image_channels']=33

    else:
        print('No Problem Type found...')
# else:
#     # load back if there is anything generated in the 1st run
#     if CKeys['Problem_ID']==2 or CKeys['Problem_ID']==6:
#         protein_df = pd.read_pickle(DataKeys['data_dir']+'protein_df.pk')
#         df_raw = pd.read_pickle(DataKeys['data_dir']+'df_raw.pk')

In [52]:
print(CKeys)

{'Running_Type': 2, 'Working_Mode': 2, 'IF_FirstRun': 2, 'Problem_ID': 11, 'Debug': 0, 'Debug_DataSet': 1, 'Debug_Model': 1, 'SlientRun': 1, 'Debug_DataPack': 0, 'Debug_ModelPack': 0, 'Debug_TrainerPack': 0, 'epochs': 750, 'print_loss_every_this_epochs': 50, 'sample_every_this_epochs': 100, 'save_model_every_this_epochs': 50, 'testratio': 0.1}


In [53]:


# ====================================================
# convert into datasets
# ====================================================
if CKeys['Working_Mode']==1 and CKeys['IF_FirstRun']==1:
    if CKeys['Problem_ID']==1:
        pass
        # train_loader, \
        # train_loader_noshuffle, \
        # test_loader, \
        # tokenizer_y, tokenizer_X = DataSetPack.load_data_set_SS_InSeqToOuSeq(
        #     file_path=SS_csv_file,
        #     PKeys=DataKeys, # to be updated
        #     CKeys=CKeys,
        # )

    elif CKeys['Problem_ID']==2:
        pass
        # train_loader, train_loader_noshuffle, \
        # test_loader, tokenizer_y, tokenizer_X = DataSetPack.load_data_set_from_df_SMD(
        #     protein_df,
        #     PKeys=DataKeys, # to be updated
        #     CKeys=CKeys,
        # )

    elif CKeys['Problem_ID']==3:
        pass
        # train_loader, train_loader_noshuffle, \
        # test_loader,tokenizer_y, tokenizer_X = DataSetPack.load_data_set_seq2seq_SecStr_ModelA (
        #     file_path=SS_csv_file, # 'PROTEIN_Mar18_2022_SECSTR_ALL.csv',
        #     PKeys=DataKeys, # to be updated
        #     CKeys=CKeys,
        # )

    elif CKeys['Problem_ID']==4:
        pass
        # train_loader, train_loader_noshuffle, \
        # test_loader,tokenizer_y, tokenizer_X = DataSetPack.load_data_set_text2seq_MD_ModelA (
        #     protein_df,
        #     PKeys=DataKeys, # to be updated
        #     CKeys=CKeys,
        # )

    # ///////////////////////////////////////////////////////////////
    #  add embedding cases
    # \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
    elif CKeys['Problem_ID']==5:
        pass
        # train_loader, \
        # train_loader_noshuffle, \
        # test_loader, \
        # tokenizer_y, \
        # tokenizer_X = DataSetPack.load_data_set_SS_InSeqToOuSeq_pLM(
        #     file_path=SS_csv_file,
        #     PKeys=DataKeys, # to be updated
        #     CKeys=CKeys,
        # )
        # # this will triger the following downloading
        # # Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /home/gridsan/bni/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
        # # excute the following if without internet on the node
        # # 1 $ wget https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt -O  /home/gridsan/bni/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
        # # 2 $ wget https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt -O /home/gridsan/bni/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt

    # add if needed
    elif CKeys['Problem_ID']==6:

        train_loader, \
        train_loader_noshuffle, \
        test_loader, \
        tokenizer_y, \
        tokenizer_X = DataSetPack.load_data_set_from_df_SMD_pLM(
                protein_df,
                PKeys=DataKeys, # to be updated
                CKeys=CKeys,
            )

    elif CKeys['Problem_ID']==7:
        pass
        # train_loader, \
        # train_loader_noshuffle, \
        # test_loader, \
        # tokenizer_y, \
        # tokenizer_X = DataSetPack.load_data_set_seq2seq_SecStr_ModelA_pLM (
        #     file_path=SS_csv_file, # 'PROTEIN_Mar18_2022_SECSTR_ALL.csv',
        #     PKeys=DataKeys, # to be updated
        #     CKeys=CKeys,
        # )

    elif CKeys['Problem_ID']==8:

        train_loader, \
        train_loader_noshuffle, \
        test_loader,\
        tokenizer_y, \
        tokenizer_X = DataSetPack.load_data_set_text2seq_MD_ModelA_pLM (
            protein_df,
            PKeys=DataKeys, # to be updated
            CKeys=CKeys,
        )

    elif CKeys['Problem_ID']==11:

        train_loader, \
        train_loader_noshuffle, \
        test_loader, \
        tokenizer_y, \
        tokenizer_X = DataSetPack.load_data_set_from_df_SMD_pLM(
                protein_df,
                PKeys=DataKeys, # to be updated
                CKeys=CKeys,
            )

    elif CKeys['Problem_ID']==12:
        pass

    else:
        print('No Problem Type found...')

    print("==========================================")
    print("Save the datasets ...")
    print("==========================================")
    # save the dataset for for the 1st run
    data_pack = {}
    data_pack['train_loader']=train_loader
    data_pack['train_loader_noshuffle']=train_loader_noshuffle
    data_pack['test_loader']=test_loader
    data_pack['tokenizer_X']=tokenizer_X
    data_pack['tokenizer_y']=tokenizer_y
    # keys
    data_pack['DataKeys']=DataKeys
    # data_pack['CKeys']=CKeys
    data_pack['PKeys']=PKeys
    with open(PKeys['pk_data_pack'], 'wb') as handle:
        pickle.dump(data_pack, handle, protocol=pickle.HIGHEST_PROTOCOL)

else: # work both for training and testing

    print('This is not the first run')
    print('Load back in the data packages...')
    with open(PKeys['pk_data_pack'], 'rb') as handle:
        data_pack = pickle.load(handle)
    # deliver the results
    train_loader=data_pack['train_loader']
    train_loader_noshuffle=data_pack['train_loader_noshuffle']
    test_loader=data_pack['test_loader']
    tokenizer_X=data_pack['tokenizer_X']
    tokenizer_y=data_pack['tokenizer_y']
    # keys (create or update)
    DataKeys=data_pack['DataKeys']
    # ++ for colab, need to update the path part
    DataKeys['data_dir']=this_working_path+'0_dataprocess_MD/'
    # ++
    # CKeys=data_pack['CKeys']
    PKeys=data_pack['PKeys']
    # ++ for colab, need to update the path part
    PKeys['prefix']=this_working_path
    PKeys['pk_data_pack']=this_working_path+'data_pack.pickle'
    PKeys['pk_model_pack']=this_working_path+'model_pack.pickle'
    # ++
    # add some for specific problem
    if CKeys['Problem_ID']==2 or CKeys['Problem_ID']==6 \
    or CKeys['Problem_ID']==11:
        pass
        # skip
        # protein_df = pd.read_pickle(DataKeys['data_dir']+'protein_df.pk')
        # df_raw = pd.read_pickle(DataKeys['data_dir']+'df_raw.pk')
    print('Done.')



This is not the first run
Load back in the data packages...
Done.


In [54]:
print(DataKeys)
print(this_working_path)

{'data_dir': '/content/working_results/0_dataprocess_MD/', 'min_AA_seq_len': 0, 'max_AA_seq_len': 128, 'max_Force_cap': 1000, 'X_Key': 'sample_FORCEpN_data', 'tokenizer_X': None, 'tokenizer_y': None, 'Xnormfac': 750.0, 'ynormfac': 1.0, 'batch_size': 256, 'testset_ratio': 0.15, 'maxdata': 99999999991000, 'ESM-2_Model': 'esm2_t30_150M_UR50D', 'image_channels': 33}
/content/working_results/


### 1.2.1 Prepare for de novo conditions

In [55]:
# add a block to handle the De Novo force shape
# idea: pick to to mix them
# if CKeys['Working_Mode']==1 and CKeys['IF_FirstRun']==1 and CKeys['Problem_ID']==2:
if CKeys['Working_Mode']==1 and CKeys['Problem_ID']==2:
    print( protein_df.keys() )
    # pick the Max_Smo_Force max and min
    pick_id_0 = protein_df['Max_Smo_Force'].idxmax()
    pick_id_1 = protein_df['Max_Smo_Force'].idxmin()
    print(protein_df['Max_Smo_Force'].idxmax())
    print(protein_df['Max_Smo_Force'].idxmin())

    fig = plt.figure(figsize=(24,16),dpi=200)
    fig, ax0 = plt.subplots()
    for ii in [pick_id_0, pick_id_1]:
        ax0.plot(
                protein_df['sample_NormPullGap_data'][ii],
                protein_df['sample_FORCE_data'][ii],
                alpha=0.1,
                # color="green",label='simplified data',
                # linestyle='None',marker='^'
            )
        ax0.scatter(
                protein_df['NPullGap_for_MaxSmoF'][ii],
                protein_df['Max_Smo_Force'][ii],
            )
    plt.xlabel('Normalized distance btw pulling ends')
    plt.ylabel('Force (pF)')
    outname = DataKeys['data_dir']+'CSV_6_PickMaxMinF_SMD_sim_Dist.jpg'
    if CKeys['SlientRun'] ==1:
        plt.savefig(outname, dpi=200)
    else:
        plt.show()
    plt.close()

elif CKeys['Working_Mode']==1 and CKeys['Problem_ID']==5:
    #
    pass
    #
elif CKeys['Working_Mode']==1 and CKeys['Problem_ID']==6:
    print( protein_df.keys() )
    # pick the Max_Smo_Force max and min
    pick_id_0 = protein_df['Max_Smo_Force'].idxmax()
    pick_id_1 = protein_df['Max_Smo_Force'].idxmin()
    # print(protein_df['Max_Smo_Force'].idxmax())
    # print(protein_df['Max_Smo_Force'].idxmin())
    print(f"Max Peak Force id: {pick_id_0}")
    print(f"Min Peak Force id: {pick_id_1}")

    fig = plt.figure(figsize=(24,16),dpi=200)
    fig, ax0 = plt.subplots()
    for ii in [pick_id_0, pick_id_1]:
        ax0.plot(
                protein_df['sample_NormPullGap_data'][ii],
                protein_df['sample_FORCE_data'][ii],
                alpha=0.1,
                # color="green",label='simplified data',
                # linestyle='None',marker='^'
            )
        ax0.scatter(
                protein_df['NPullGap_for_MaxSmoF'][ii],
                protein_df['Max_Smo_Force'][ii],
            )
    plt.xlabel('Normalized distance btw pulling ends')
    plt.ylabel('Force (pF)')
    outname = DataKeys['data_dir']+'CSV_6_PickMaxMinF_SMD_sim_Dist.jpg'
    if CKeys['SlientRun'] ==1:
        plt.savefig(outname, dpi=200)
    else:
        plt.show()
    plt.close()

elif CKeys['Working_Mode']==1 and CKeys['Problem_ID']==11:
    # skipped
    pass
    # print( protein_df.keys() )
    # # pick the Max_Smo_Force max and min
    # pick_id_0 = protein_df['Max_Smo_Force'].idxmax()
    # pick_id_1 = protein_df['Max_Smo_Force'].idxmin()
    # # print(protein_df['Max_Smo_Force'].idxmax())
    # # print(protein_df['Max_Smo_Force'].idxmin())
    # print(f"Max Peak Force id: {pick_id_0}")
    # print(f"Min Peak Force id: {pick_id_1}")

    # fig = plt.figure(figsize=(24,16),dpi=200)
    # fig, ax0 = plt.subplots()
    # for ii in [pick_id_0, pick_id_1]:
    #     ax0.plot(
    #             protein_df['sample_NormPullGap_data'][ii],
    #             protein_df['sample_FORCE_data'][ii],
    #             alpha=0.1,
    #             # color="green",label='simplified data',
    #             # linestyle='None',marker='^'
    #         )
    #     ax0.scatter(
    #             protein_df['NPullGap_for_MaxSmoF'][ii],
    #             protein_df['Max_Smo_Force'][ii],
    #         )
    # plt.xlabel('Normalized distance btw pulling ends')
    # plt.ylabel('Force (pF)')
    # outname = DataKeys['data_dir']+'CSV_6_PickMaxMinF_SMD_sim_Dist.jpg'
    # if CKeys['SlientRun'] ==1:
    #     plt.savefig(outname, dpi=200)
    # else:
    #     plt.show()
    # plt.close()

In [56]:
# a quick check
print('tokenizer_X: ', tokenizer_X)
print('tokenizer_y: ', tokenizer_y)
print('Xnormfac: ', DataKeys['Xnormfac'])
print('ynormfac: ', DataKeys['ynormfac'])


tokenizer_X:  None
tokenizer_y:  None
Xnormfac:  750.0
ynormfac:  1.0


In [57]:
if CKeys['Debug']==1:

    if CKeys['Debug_DataPack']==1:
        # uppack to check
        print('Len of train loader:', len(train_loader))
        print('Len of test loader:', len(test_loader))
        ii=-1
        # for item in train_loader:
        for item in test_loader:
            ii += 1
            if ii<1:
                print('Len of 1st batch item: ', len(item))
                this_item = item
            else:
                pass

        # on mini-batch
        print('Batch size: ', DataKeys['batch_size'])
        print('Seq len: ', DataKeys['max_AA_seq_len'])
        print('X.dim: ', this_item[0].shape) # Condition: (Batch, Condi)
        print('y_data.dim: ', this_item[1].shape) # AASequenc: (Batch, AACode)
        #
        # # print(this_item[0][0,:]*DataKeys['Xnormfac'])
        # # print(this_item[1][0,:]*DataKeys['ynormfac'])
        print(this_item[0][0,:])
        # print(torch.FloatTensor(DataKeys['Xnormfac']))
        # print(this_item[0][0,:]*torch.FloatTensor(DataKeys['Xnormfac']))


In [58]:
if CKeys['Debug']==1:

    if CKeys['Debug_DataPack']==1:

        if CKeys['Problem_ID']==8 or CKeys['Problem_ID']==7:

            print (this_item[0][0,:])
            print (this_item[0][0,:]*DataKeys['Xnormfac'])

### 1.3 Model building

In [59]:
# import PD_pLMProbXDiff.ModelPack as ModelPack
# #
# import PD_pLMProbXDiff.TrainerPack as TrainerPack
# #
# importlib.reload(ModelPack)
# importlib.reload(TrainerPack)

In [60]:

if CKeys['Working_Mode']==1 and CKeys['IF_FirstRun']==1:
    # this is a trining mode
    if CKeys['Problem_ID']==1:
        pass
#         # this is 1st run....
#         # +++++++++++++++++++++++++++++++++++++
#         # SecStr as input seq
#         ModelKeys={}
#         # storage
#         ModelKeys['model_dir']=PKey['prefix']+'1_model_SS/'
#         # create the folder
#         print("Creating the model dir...")
#         UtilityPack.create_path(ModelKeys['model_dir'])
#         # secondary folders
#         ModelKeys['model_dir_sample']=ModelKeys['model_dir']+'0_mid_sample/'
#         UtilityPack.create_path(ModelKeys['model_dir_sample'])
#         ModelKeys['model_dir_model']=ModelKeys['model_dir']+'1_store_model/'
#         UtilityPack.create_path(ModelKeys['model_dir_model'])

#         # for UNet
#         ModelKeys['dim']=256    # dim for UNet
#         ModelKeys['text_embed_dim'] = 512
#         ModelKeys['embed_dim_position']=32
#         ModelKeys['text_embed_dim']=ModelKeys['text_embed_dim']+ModelKeys['embed_dim_position']
#         ModelKeys['cond_dim'] = 512
#         ModelKeys['cond_images_channels']=1
#         ModelKeys['max_text_len']=DataKeys['max_AA_seq_len'] # this is about text condi
#         # for Imagen
#         ModelKeys['pred_dim']=1 # for sequence, =1
#         ModelKeys['diff_timesteps']=(96,)
#         ModelKeys['loss_type']=0 # MSE
#         ModelKeys['elucidated']=True #
#         ModelKeys['padding_idx']=0
#         ModelKeys['max_length']=DataKeys['max_AA_seq_len'] # max seq len
#         ModelKeys['device']=device
#         #
#         # buckets for Model building
#         ModelKeys['UNet']={}
#         ModelKeys['Imagen']={}
#         # ++++++++++++++++++++++++++++++++++++++++++++++++++
#         write_PK_UNet=dict()
#         write_PK_UNet['dim']=ModelKeys['dim'] # 256
#         write_PK_UNet['text_embed_dim']=ModelKeys['text_embed_dim'] # 512+32
#         write_PK_UNet['num_resnet_blocks']=1
#         write_PK_UNet['cond_dim']=ModelKeys['cond_dim']  #this is where text embeddings are projected to...
#         # write_PK_UNet['num_image_tokens']=None # using the default value
#         # write_PK_UNet['num_time_tokens']=None  # using the default
#         # write_PK_UNet['learned_sinu_pos_emb_dim']=None
#         # write_PK_UNet['out_dim']=None
#         write_PK_UNet['dim_mults']=(1, 2, 4, 8)

#         write_PK_UNet['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_UNet['channels']=1
#         write_PK_UNet['channels_out']=1

#         write_PK_UNet['attn_dim_head']=64
#         write_PK_UNet['attn_heads']=8
#         write_PK_UNet['ff_mult']=2.
#         write_PK_UNet['lowres_cond']=False  # for cascading diffusion - https://cascaded-diffusion.github.io/

#         write_PK_UNet['layer_attns']=(False, True, True, False)
#         write_PK_UNet['layer_attns_depth']=1
#         write_PK_UNet['layer_attns_add_text_cond']=True # whether to condition the self-attention blocks with the text embeddings, as described in Appendix D.3.1
#         write_PK_UNet['attend_at_middle']=True          # whether to have a layer of attention at the bottleneck (can turn off for higher resolution in cascading DDPM, before bringing in efficient attention)
#         write_PK_UNet['layer_cross_attns']=(False, True, True, False)
#         write_PK_UNet['use_linear_attn']=False
#         write_PK_UNet['use_linear_cross_attn']=False

#         write_PK_UNet['cond_on_text']=True
#         write_PK_UNet['max_text_len']=ModelKeys['max_text_len'] # need to check this one
#         # ModelKeys['UNet']['init_dim']=None
#         write_PK_UNet['resnet_groups']=8
#         write_PK_UNet['init_conv_kernel_size']=7 # kernel size of initial conv, if not using cross embed
#         write_PK_UNet['init_cross_embed']=False  #TODO - fix ouput size calcs for conv1d
#         write_PK_UNet['init_cross_embed_kernel_sizes']=(3, 7, 15)
#         write_PK_UNet['cross_embed_downsample']=False
#         write_PK_UNet['cross_embed_downsample_kernel_sizes']=(2, 4)

#         write_PK_UNet['attn_pool_text']=True
#         write_PK_UNet['attn_pool_num_latents']=32 #perceiver model latents
#         write_PK_UNet['dropout']=0.
#         write_PK_UNet['memory_efficient']=False
#         write_PK_UNet['init_conv_to_final_conv_residual']=False

#         write_PK_UNet['use_global_context_attn']=True
#         write_PK_UNet['scale_skip_connection']=True
#         write_PK_UNet['final_resnet_block']=True
#         write_PK_UNet['final_conv_kernel_size']=3

#         write_PK_UNet['cosine_sim_attn']=True
#         write_PK_UNet['self_cond']=False
#         write_PK_UNet['combine_upsample_fmaps']=True    # combine feature maps from all upsample blocks, used in unet squared successfully
#         write_PK_UNet['pixel_shuffle_upsample']=False   # may address checkboard artifacts
#         # write_PK_UNet['beginning_and_final_conv_present']=None # use default
#         ModelKeys['UNet']=UtilityPack.prepare_UNet_keys(write_PK_UNet)
#         # +++++++++++++++++++++++++++++++++++++++++++++++++++++
#         # beyond UNet, for Whole model, all keys
#         # "None" means defult value on function definition
#         write_PK_Imagen=dict()

#         write_PK_Imagen['timesteps']=ModelKeys['diff_timesteps']
#         write_PK_Imagen['dim']=ModelKeys['dim']
#         write_PK_Imagen['pred_dim']=ModelKeys['pred_dim']
#         write_PK_Imagen['loss_type']=ModelKeys['loss_type'] # 0 # MSE
#         write_PK_Imagen['elucidated']=ModelKeys['elucidated'] # True
#         write_PK_Imagen['padding_idx']=ModelKeys['padding_idx'] # 0 # need to check
#         # write_PK_Imagen['cond_dim']=None # use default
#         # write_PK_Imagen['text_embed_dim']=None
#         # write_PK_Imagen['input_tokens']=None
#         # write_PK_Imagen['sequence_embed']=None
#         # write_PK_Imagen['embed_dim_position']=None
#         write_PK_Imagen['max_text_len']=ModelKeys['max_text_len']
#         write_PK_Imagen['cond_images_channels']=ModelKeys['pred_dim']
#         write_PK_Imagen['max_length']=ModelKeys['max_length']
#         write_PK_Imagen['device']=ModelKeys['device']

#         # extend it to a full key
#         ModelKeys['Imagen']=UtilityPack.prepare_ModelB_keys(write_PK_Imagen)

    elif CKeys['Problem_ID']==2:
        pass
#         # this is 1st run....
#         # +++++++++++++++++++++++++++++++++++++
#         # SecStr as input seq
#         ModelKeys={}
#         # storage
#         ModelKeys['model_dir']=PKeys['prefix']+'1_model_SS/'
#         # create the folder
#         print("Creating the model dir...")
#         UtilityPack.create_path(ModelKeys['model_dir'])
#         # secondary folders
#         ModelKeys['model_dir_sample']=ModelKeys['model_dir']+'0_mid_sample/'
#         UtilityPack.create_path(ModelKeys['model_dir_sample'])
#         ModelKeys['model_dir_model']=ModelKeys['model_dir']+'1_store_model/'
#         UtilityPack.create_path(ModelKeys['model_dir_model'])

#         # for UNet
#         ModelKeys['dim']=256    # dim for UNet
#         ModelKeys['text_embed_dim'] = 512
#         ModelKeys['embed_dim_position']=32
#         ModelKeys['text_embed_dim']=ModelKeys['text_embed_dim']+ModelKeys['embed_dim_position']
#         ModelKeys['cond_dim'] = 512
#         ModelKeys['cond_images_channels']=1
#         ModelKeys['max_text_len']=DataKeys['max_AA_seq_len'] # this is about text condi
#         # for Imagen
#         ModelKeys['pred_dim']=1 # for sequence, =1
#         ModelKeys['diff_timesteps']=(96,)
#         ModelKeys['loss_type']=0 # MSE
#         ModelKeys['elucidated']=True #
#         ModelKeys['padding_idx']=0
#         ModelKeys['max_length']=DataKeys['max_AA_seq_len'] # max seq len
#         ModelKeys['device']=device
#         #
#         # buckets for Model building
#         ModelKeys['UNet']={}
#         ModelKeys['Imagen']={}
#         # ++++++++++++++++++++++++++++++++++++++++++++++++++
#         write_PK_UNet=dict()
#         write_PK_UNet['dim']=ModelKeys['dim'] # 256
#         write_PK_UNet['text_embed_dim']=ModelKeys['text_embed_dim'] # 512+32
#         write_PK_UNet['num_resnet_blocks']=1
#         write_PK_UNet['cond_dim']=ModelKeys['cond_dim']  #this is where text embeddings are projected to...
#         # write_PK_UNet['num_image_tokens']=None # using the default value
#         # write_PK_UNet['num_time_tokens']=None  # using the default
#         # write_PK_UNet['learned_sinu_pos_emb_dim']=None
#         # write_PK_UNet['out_dim']=None
#         write_PK_UNet['dim_mults']=(1, 2, 4, 8)

#         write_PK_UNet['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_UNet['channels']=1
#         write_PK_UNet['channels_out']=1

#         write_PK_UNet['attn_dim_head']=64
#         write_PK_UNet['attn_heads']=8
#         write_PK_UNet['ff_mult']=2.
#         write_PK_UNet['lowres_cond']=False  # for cascading diffusion - https://cascaded-diffusion.github.io/

#         write_PK_UNet['layer_attns']=(False, True, True, False)
#         write_PK_UNet['layer_attns_depth']=1
#         write_PK_UNet['layer_attns_add_text_cond']=True # whether to condition the self-attention blocks with the text embeddings, as described in Appendix D.3.1
#         write_PK_UNet['attend_at_middle']=True          # whether to have a layer of attention at the bottleneck (can turn off for higher resolution in cascading DDPM, before bringing in efficient attention)
#         write_PK_UNet['layer_cross_attns']=(False, True, True, False)
#         write_PK_UNet['use_linear_attn']=False
#         write_PK_UNet['use_linear_cross_attn']=False

#         write_PK_UNet['cond_on_text']=True
#         write_PK_UNet['max_text_len']=ModelKeys['max_text_len'] # need to check this one
#         # ModelKeys['UNet']['init_dim']=None
#         write_PK_UNet['resnet_groups']=8
#         write_PK_UNet['init_conv_kernel_size']=7 # kernel size of initial conv, if not using cross embed
#         write_PK_UNet['init_cross_embed']=False  #TODO - fix ouput size calcs for conv1d
#         write_PK_UNet['init_cross_embed_kernel_sizes']=(3, 7, 15)
#         write_PK_UNet['cross_embed_downsample']=False
#         write_PK_UNet['cross_embed_downsample_kernel_sizes']=(2, 4)

#         write_PK_UNet['attn_pool_text']=True
#         write_PK_UNet['attn_pool_num_latents']=32 #perceiver model latents
#         write_PK_UNet['dropout']=0.
#         write_PK_UNet['memory_efficient']=False
#         write_PK_UNet['init_conv_to_final_conv_residual']=False

#         write_PK_UNet['use_global_context_attn']=True
#         write_PK_UNet['scale_skip_connection']=True
#         write_PK_UNet['final_resnet_block']=True
#         write_PK_UNet['final_conv_kernel_size']=3

#         write_PK_UNet['cosine_sim_attn']=True
#         write_PK_UNet['self_cond']=False
#         write_PK_UNet['combine_upsample_fmaps']=True    # combine feature maps from all upsample blocks, used in unet squared successfully
#         write_PK_UNet['pixel_shuffle_upsample']=False   # may address checkboard artifacts
#         # write_PK_UNet['beginning_and_final_conv_present']=None # use default
#         ModelKeys['UNet']=UtilityPack.prepare_UNet_keys(write_PK_UNet)
#         # +++++++++++++++++++++++++++++++++++++++++++++++++++++
#         # beyond UNet, for Whole model, all keys
#         # "None" means defult value on function definition
#         write_PK_Imagen=dict()

#         write_PK_Imagen['timesteps']=ModelKeys['diff_timesteps']
#         write_PK_Imagen['dim']=ModelKeys['dim']
#         write_PK_Imagen['pred_dim']=ModelKeys['pred_dim']
#         write_PK_Imagen['loss_type']=ModelKeys['loss_type'] # 0 # MSE
#         write_PK_Imagen['elucidated']=ModelKeys['elucidated'] # True
#         write_PK_Imagen['padding_idx']=ModelKeys['padding_idx'] # 0 # need to check
#         # write_PK_Imagen['cond_dim']=None # use default
#         # write_PK_Imagen['text_embed_dim']=None
#         # write_PK_Imagen['input_tokens']=None
#         # write_PK_Imagen['sequence_embed']=None
#         # write_PK_Imagen['embed_dim_position']=None
#         write_PK_Imagen['max_text_len']=ModelKeys['max_text_len']
#         write_PK_Imagen['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_Imagen['max_length']=ModelKeys['max_length']
#         write_PK_Imagen['device']=ModelKeys['device']

#         # extend it to a full key
#         ModelKeys['Imagen']=UtilityPack.prepare_ModelB_keys(write_PK_Imagen)
#         # to be defined

    elif CKeys['Problem_ID']==3:
        pass

#         ModelKeys={}
#         # storage
#         ModelKeys['model_dir']=PKeys['prefix']+'1_model_SS/'
#         # create the folder
#         print("Creating the model dir...")
#         UtilityPack.create_path(ModelKeys['model_dir'])
#         # secondary folders
#         ModelKeys['model_dir_sample']=ModelKeys['model_dir']+'0_mid_sample/'
#         UtilityPack.create_path(ModelKeys['model_dir_sample'])
#         ModelKeys['model_dir_model']=ModelKeys['model_dir']+'1_store_model/'
#         UtilityPack.create_path(ModelKeys['model_dir_model'])

#         # for UNet
#         ModelKeys['dim']=768    # dim for UNet
#         # for debug
#         ModelKeys['dim']=256    # dim for UNet: GPU stat: 256:16093MiB;
#         #
#         ModelKeys['text_embed_dim']=512-128
#         ModelKeys['embed_dim_position']=128
#             # ! this is for UNet, not for Imagen
#         ModelKeys['text_embed_dim']=ModelKeys['text_embed_dim']+ModelKeys['embed_dim_position']
#         ModelKeys['cond_dim']=512
#         # ModelKeys['embed_dim_position']=128
#         # ModelKeys['text_embed_dim']=ModelKeys['cond_dim']-ModelKeys['embed_dim_position']
#         # !!! Need to check this one = 1 or not
#         ModelKeys['cond_images_channels']=0 # 1
#         # !!! Need to check this one: should it be 8 or 64?
#         ModelKeys['max_text_len']= DataKeys['max_text_len'] # DataKeys['max_AA_seq_len'] # 8 # this is about text condi

#         # for Imagen
#         ModelKeys['pred_dim']=1 # for sequence, =1
#         ModelKeys['diff_timesteps']=(96) # (96,)
#         ModelKeys['loss_type']=0 # MSE
#         ModelKeys['elucidated']=True #
#         ModelKeys['padding_idx']=0
#         ModelKeys['max_length']=DataKeys['max_AA_seq_len']
#         ModelKeys['device']=device
#         #
#         # baskets for Model building
#         ModelKeys['UNet']={}
#         ModelKeys['Imagen']={}
#         # ++++++++++++++++++++++++++++++++++++++
#         # prepare the Unet Key
#         write_PK_UNet=dict()
#         # used ones
#         write_PK_UNet['dim']=ModelKeys['dim']
#         write_PK_UNet['text_embed_dim']=ModelKeys['text_embed_dim']
#         write_PK_UNet['num_resnet_blocks']=1
#         write_PK_UNet['cond_dim']=ModelKeys['cond_dim'] #this is where text embeddings are projected to...
#          # write_PK_UNet['num_image_tokens']=None # using the default value
#         # write_PK_UNet['num_time_tokens']=None  # using the default
#         # write_PK_UNet['learned_sinu_pos_emb_dim']=None
#         # write_PK_UNet['out_dim']=None
#         write_PK_UNet['dim_mults']=(1, 2, 4, 8)

#         write_PK_UNet['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_UNet['channels']=ModelKeys['pred_dim']
#         write_PK_UNet['channels_out']=ModelKeys['pred_dim']

#         write_PK_UNet['attn_dim_head']=64
#         write_PK_UNet['attn_heads']=8
#         write_PK_UNet['ff_mult']=2.
#         write_PK_UNet['lowres_cond']=False # for cascading diffusion - https://cascaded-diffusion.github.io/


#         write_PK_UNet['layer_attns']=(False, True, True, False)
#         write_PK_UNet['layer_attns_depth']=1
#         write_PK_UNet['layer_attns_add_text_cond']=True # whether to condition the self-attention blocks with the text embeddings, as described in Appendix D.3.1
#         write_PK_UNet['attend_at_middle']=True # whether to have a layer of attention at the bottleneck (can turn off for higher resolution in cascading DDPM, before bringing in efficient attention)
#         write_PK_UNet['layer_cross_attns']=(False, True, True, False)
#         write_PK_UNet['use_linear_attn']=False
#         write_PK_UNet['use_linear_cross_attn']=False

#         write_PK_UNet['cond_on_text'] = True
#             # !!! Need to check this: 63, Imagen used a different one
#         write_PK_UNet['max_text_len'] = DataKeys['max_AA_seq_len'] # ModelKeys['max_text_len']
#         # write_PK_UNet['init_dim'] = None
#         write_PK_UNet['resnet_groups'] = 8
#         write_PK_UNet['init_conv_kernel_size'] =7     # kernel size of initial conv, if not using cross embed
#         write_PK_UNet['init_cross_embed'] = False  #TODO - fix ouput size calcs for conv1d
#         write_PK_UNet['init_cross_embed_kernel_sizes'] = (3, 7, 15)
#         write_PK_UNet['cross_embed_downsample'] = False
#         write_PK_UNet['cross_embed_downsample_kernel_sizes'] = (2, 4)

#         write_PK_UNet['attn_pool_text'] = True
#         write_PK_UNet['attn_pool_num_latents'] = 32 #32,   #perceiver model latents
#         write_PK_UNet['dropout'] = 0.
#         write_PK_UNet['memory_efficient'] = False
#         write_PK_UNet['init_conv_to_final_conv_residual'] = False

#         write_PK_UNet['use_global_context_attn'] = True
#         write_PK_UNet['scale_skip_connection'] = True
#         write_PK_UNet['final_resnet_block'] = True
#         write_PK_UNet['final_conv_kernel_size'] = 3

#         write_PK_UNet['cosine_sim_attn'] = True
#         write_PK_UNet['self_cond'] = False
#         write_PK_UNet['combine_upsample_fmaps'] = True      # combine feature maps from all upsample blocks, used in unet squared successfully
#         write_PK_UNet['pixel_shuffle_upsample'] = False     # may address checkboard artifacts
#         # write_PK_UNet['beginning_and_final_conv_present']=None # use default
#         #
#         ModelKeys['UNet']=UtilityPack.prepare_UNet_keys(write_PK_UNet)
#         # ++++++++++++++++++++++++++++++++++++++
#         # beyond UNet, for Whole model, all keys
#         # "None" means defult value on function definition
#         write_PK_Imagen=dict()

#         # used ones
#         write_PK_Imagen['timesteps']=ModelKeys['diff_timesteps']
#         write_PK_Imagen['dim']=ModelKeys['dim']
#         write_PK_Imagen['pred_dim']=ModelKeys['pred_dim']
#         write_PK_Imagen['loss_type']=ModelKeys['loss_type'] # 0 # MSE
#         write_PK_Imagen['elucidated']=ModelKeys['elucidated'] # True
#         write_PK_Imagen['padding_idx']=ModelKeys['padding_idx'] # 0 # need to check
#         write_PK_Imagen['cond_dim']=ModelKeys['cond_dim'] # use default
#             # can use default, 512; or like the below (check the code)
#         write_PK_Imagen['text_embed_dim']=ModelKeys['text_embed_dim']-ModelKeys['embed_dim_position']
#         # write_PK_Imagen['input_tokens']=None
#         # write_PK_Imagen['sequence_embed']=None
#         write_PK_Imagen['embed_dim_position']=ModelKeys['embed_dim_position']
#         write_PK_Imagen['max_text_len']=ModelKeys['max_text_len']
#         write_PK_Imagen['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_Imagen['max_length']=ModelKeys['max_length']
#         write_PK_Imagen['device']=ModelKeys['device']

#         # extend it to a full key
#         ModelKeys['Imagen']=UtilityPack.prepare_ModelB_keys(write_PK_Imagen)

    elif CKeys['Problem_ID']==4:
        pass

#         ModelKeys={}
#         # storage
#         ModelKeys['model_dir']=PKeys['prefix']+'1_model_SS/'
#         # create the folder
#         print("Creating the model dir...")
#         UtilityPack.create_path(ModelKeys['model_dir'])
#         # secondary folders
#         ModelKeys['model_dir_sample']=ModelKeys['model_dir']+'0_mid_sample/'
#         UtilityPack.create_path(ModelKeys['model_dir_sample'])
#         ModelKeys['model_dir_model']=ModelKeys['model_dir']+'1_store_model/'
#         UtilityPack.create_path(ModelKeys['model_dir_model'])

#         # for UNet
#         ModelKeys['dim']=768    # dim for UNet
#         # for debug
#         ModelKeys['dim']=256    # dim for UNet: GPU stat: 256:16093MiB;
#         #
#         ModelKeys['text_embed_dim']=512-128
#         ModelKeys['embed_dim_position']=128
#             # ! this is for UNet, not for Imagen
#         ModelKeys['text_embed_dim']=ModelKeys['text_embed_dim']+ModelKeys['embed_dim_position']
#         ModelKeys['cond_dim']=512
#         # ModelKeys['embed_dim_position']=128
#         # ModelKeys['text_embed_dim']=ModelKeys['cond_dim']-ModelKeys['embed_dim_position']
#         # !!! Need to check this one = 1 or not
#         ModelKeys['cond_images_channels']=0 # 1
#         # !!! Need to check this one: should it be 8 or 64?
#         ModelKeys['max_text_len']= DataKeys['max_text_len'] # DataKeys['max_AA_seq_len'] # 8 # this is about text condi

#         # for Imagen
#         ModelKeys['pred_dim']=1 # for sequence, =1
#         ModelKeys['diff_timesteps']=(96) # (96,)
#         ModelKeys['loss_type']=0 # MSE
#         ModelKeys['elucidated']=True #
#         ModelKeys['padding_idx']=0
#         ModelKeys['max_length']=DataKeys['max_AA_seq_len']
#         ModelKeys['device']=device
#         #
#         # baskets for Model building
#         ModelKeys['UNet']={}
#         ModelKeys['Imagen']={}
#         # ++++++++++++++++++++++++++++++++++++++
#         # prepare the Unet Key
#         write_PK_UNet=dict()
#         # used ones
#         write_PK_UNet['dim']=ModelKeys['dim']
#         write_PK_UNet['text_embed_dim']=ModelKeys['text_embed_dim']
#         write_PK_UNet['num_resnet_blocks']=1
#         write_PK_UNet['cond_dim']=ModelKeys['cond_dim'] #this is where text embeddings are projected to...
#          # write_PK_UNet['num_image_tokens']=None # using the default value
#         # write_PK_UNet['num_time_tokens']=None  # using the default
#         # write_PK_UNet['learned_sinu_pos_emb_dim']=None
#         # write_PK_UNet['out_dim']=None
#         write_PK_UNet['dim_mults']=(1, 2, 4, 8)

#         write_PK_UNet['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_UNet['channels']=ModelKeys['pred_dim']
#         write_PK_UNet['channels_out']=ModelKeys['pred_dim']

#         write_PK_UNet['attn_dim_head']=64
#         write_PK_UNet['attn_heads']=8
#         write_PK_UNet['ff_mult']=2.
#         write_PK_UNet['lowres_cond']=False # for cascading diffusion - https://cascaded-diffusion.github.io/


#         write_PK_UNet['layer_attns']=(False, True, True, False)
#         write_PK_UNet['layer_attns_depth']=1
#         write_PK_UNet['layer_attns_add_text_cond']=True # whether to condition the self-attention blocks with the text embeddings, as described in Appendix D.3.1
#         write_PK_UNet['attend_at_middle']=True # whether to have a layer of attention at the bottleneck (can turn off for higher resolution in cascading DDPM, before bringing in efficient attention)
#         write_PK_UNet['layer_cross_attns']=(False, True, True, False)
#         write_PK_UNet['use_linear_attn']=False
#         write_PK_UNet['use_linear_cross_attn']=False

#         write_PK_UNet['cond_on_text'] = True
#             # !!! Need to check this: 63, Imagen used a different one
#         write_PK_UNet['max_text_len'] = DataKeys['max_text_len'] # DataKeys['max_AA_seq_len'] # ModelKeys['max_text_len']
#         # write_PK_UNet['init_dim'] = None
#         write_PK_UNet['resnet_groups'] = 8
#         write_PK_UNet['init_conv_kernel_size'] =7     # kernel size of initial conv, if not using cross embed
#         write_PK_UNet['init_cross_embed'] = False  #TODO - fix ouput size calcs for conv1d
#         write_PK_UNet['init_cross_embed_kernel_sizes'] = (3, 7, 15)
#         write_PK_UNet['cross_embed_downsample'] = False
#         write_PK_UNet['cross_embed_downsample_kernel_sizes'] = (2, 4)

#         write_PK_UNet['attn_pool_text'] = True
#         write_PK_UNet['attn_pool_num_latents'] = 32 #32,   #perceiver model latents
#         write_PK_UNet['dropout'] = 0.
#         write_PK_UNet['memory_efficient'] = False
#         write_PK_UNet['init_conv_to_final_conv_residual'] = False

#         write_PK_UNet['use_global_context_attn'] = True
#         write_PK_UNet['scale_skip_connection'] = True
#         write_PK_UNet['final_resnet_block'] = True
#         write_PK_UNet['final_conv_kernel_size'] = 3

#         write_PK_UNet['cosine_sim_attn'] = True
#         write_PK_UNet['self_cond'] = False
#         write_PK_UNet['combine_upsample_fmaps'] = True      # combine feature maps from all upsample blocks, used in unet squared successfully
#         write_PK_UNet['pixel_shuffle_upsample'] = False     # may address checkboard artifacts
#         # write_PK_UNet['beginning_and_final_conv_present']=None # use default
#         #
#         ModelKeys['UNet']=UtilityPack.prepare_UNet_keys(write_PK_UNet)
#         # ++++++++++++++++++++++++++++++++++++++
#         # beyond UNet, for Whole model, all keys
#         # "None" means defult value on function definition
#         write_PK_Imagen=dict()

#         # used ones
#         write_PK_Imagen['timesteps']=ModelKeys['diff_timesteps']
#         write_PK_Imagen['dim']=ModelKeys['dim']
#         write_PK_Imagen['pred_dim']=ModelKeys['pred_dim']
#         write_PK_Imagen['loss_type']=ModelKeys['loss_type'] # 0 # MSE
#         write_PK_Imagen['elucidated']=ModelKeys['elucidated'] # True
#         write_PK_Imagen['padding_idx']=ModelKeys['padding_idx'] # 0 # need to check
#         write_PK_Imagen['cond_dim']=ModelKeys['cond_dim'] # use default
#             # can use default, 512; or like the below (check the code)
#         write_PK_Imagen['text_embed_dim']=ModelKeys['text_embed_dim']-ModelKeys['embed_dim_position']
#         # write_PK_Imagen['input_tokens']=None
#         # write_PK_Imagen['sequence_embed']=None
#         write_PK_Imagen['embed_dim_position']=ModelKeys['embed_dim_position']
#         write_PK_Imagen['max_text_len']=ModelKeys['max_text_len']
#         write_PK_Imagen['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_Imagen['max_length']=ModelKeys['max_length']
#         write_PK_Imagen['device']=ModelKeys['device']

#         # extend it to a full key
#         ModelKeys['Imagen']=UtilityPack.prepare_ModelB_keys(write_PK_Imagen)

    elif CKeys['Problem_ID']==5:
        pass

#         # this is 1st run....
#         # +++++++++++++++++++++++++++++++++++++
#         # SecStr as input seq
#         ModelKeys={}
#         # storage
#         ModelKeys['model_dir']=PKeys['prefix']+'1_model_SS/'
#         # create the folder
#         print("Creating the model dir...")
#         UtilityPack.create_path(ModelKeys['model_dir'])
#         # secondary folders
#         ModelKeys['model_dir_sample']=ModelKeys['model_dir']+'0_mid_sample/'
#         UtilityPack.create_path(ModelKeys['model_dir_sample'])
#         ModelKeys['model_dir_model']=ModelKeys['model_dir']+'1_store_model/'
#         UtilityPack.create_path(ModelKeys['model_dir_model'])

#         # for UNet
#         ModelKeys['dim']=256    # dim for UNet
#         ModelKeys['text_embed_dim'] = 512
#         ModelKeys['embed_dim_position']=32
#         ModelKeys['text_embed_dim']=ModelKeys['text_embed_dim']+ModelKeys['embed_dim_position']
#         ModelKeys['cond_dim'] = 512
#         ModelKeys['cond_images_channels']=DataKeys['image_channels'] # 1280 # for embedding dim # 1
#         ModelKeys['max_text_len']=DataKeys['max_AA_seq_len'] # this is about text condi
#         # + for embedding
#         ModelKeys['image_channels']=DataKeys['image_channels'] # 1280
#         # for Imagen
#         ModelKeys['pred_dim']= ModelKeys['image_channels'] # 1 # for sequence, =1
#         ModelKeys['diff_timesteps']=(96,)
#         ModelKeys['loss_type']=0 # MSE
#         ModelKeys['elucidated']=True #
#         ModelKeys['padding_idx']=0
#         ModelKeys['max_length']=DataKeys['max_AA_seq_len'] # max seq len
#         ModelKeys['device']=device
#         #
#         # buckets for Model building
#         ModelKeys['UNet']={}
#         ModelKeys['Imagen']={}
#         # ++++++++++++++++++++++++++++++++++++++++++++++++++
#         write_PK_UNet=dict()
#         write_PK_UNet['dim']=ModelKeys['dim'] # 256
#         write_PK_UNet['text_embed_dim']=ModelKeys['text_embed_dim'] # 512+32
#         write_PK_UNet['num_resnet_blocks']=1
#         write_PK_UNet['cond_dim']=ModelKeys['cond_dim']  #this is where text embeddings are projected to...
#         # write_PK_UNet['num_image_tokens']=None # using the default value
#         # write_PK_UNet['num_time_tokens']=None  # using the default
#         # write_PK_UNet['learned_sinu_pos_emb_dim']=None
#         # write_PK_UNet['out_dim']=None
#         write_PK_UNet['dim_mults']=(1, 2, 4, 8)

#         write_PK_UNet['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_UNet['channels']=ModelKeys['image_channels'] # 1
#         write_PK_UNet['channels_out']=ModelKeys['image_channels'] # 1

#         write_PK_UNet['attn_dim_head']=64
#         write_PK_UNet['attn_heads']=8
#         write_PK_UNet['ff_mult']=2.
#         write_PK_UNet['lowres_cond']=False  # for cascading diffusion - https://cascaded-diffusion.github.io/

#         write_PK_UNet['layer_attns']=(False, True, True, False)
#         write_PK_UNet['layer_attns_depth']=1
#         write_PK_UNet['layer_attns_add_text_cond']=True # whether to condition the self-attention blocks with the text embeddings, as described in Appendix D.3.1
#         write_PK_UNet['attend_at_middle']=True          # whether to have a layer of attention at the bottleneck (can turn off for higher resolution in cascading DDPM, before bringing in efficient attention)
#         write_PK_UNet['layer_cross_attns']=(False, True, True, False)
#         write_PK_UNet['use_linear_attn']=False
#         write_PK_UNet['use_linear_cross_attn']=False

#         write_PK_UNet['cond_on_text']=True
#         write_PK_UNet['max_text_len']=ModelKeys['max_text_len'] # need to check this one
#         # ModelKeys['UNet']['init_dim']=None
#         write_PK_UNet['resnet_groups']=8
#         write_PK_UNet['init_conv_kernel_size']=7 # kernel size of initial conv, if not using cross embed
#         write_PK_UNet['init_cross_embed']=False  #TODO - fix ouput size calcs for conv1d
#         write_PK_UNet['init_cross_embed_kernel_sizes']=(3, 7, 15)
#         write_PK_UNet['cross_embed_downsample']=False
#         write_PK_UNet['cross_embed_downsample_kernel_sizes']=(2, 4)

#         write_PK_UNet['attn_pool_text']=True
#         write_PK_UNet['attn_pool_num_latents']=32 #perceiver model latents
#         write_PK_UNet['dropout']=0.
#         write_PK_UNet['memory_efficient']=False
#         write_PK_UNet['init_conv_to_final_conv_residual']=False

#         write_PK_UNet['use_global_context_attn']=True
#         write_PK_UNet['scale_skip_connection']=True
#         write_PK_UNet['final_resnet_block']=True
#         write_PK_UNet['final_conv_kernel_size']=3

#         write_PK_UNet['cosine_sim_attn']=True
#         write_PK_UNet['self_cond']=False
#         write_PK_UNet['combine_upsample_fmaps']=True    # combine feature maps from all upsample blocks, used in unet squared successfully
#         write_PK_UNet['pixel_shuffle_upsample']=False   # may address checkboard artifacts
#         # write_PK_UNet['beginning_and_final_conv_present']=None # use default
#         ModelKeys['UNet']=UtilityPack.prepare_UNet_keys(write_PK_UNet)
#         # +++++++++++++++++++++++++++++++++++++++++++++++++++++
#         # beyond UNet, for Whole model, all keys
#         # "None" means defult value on function definition
#         write_PK_Imagen=dict()

#         write_PK_Imagen['timesteps']=ModelKeys['diff_timesteps']
#         write_PK_Imagen['dim']=ModelKeys['dim']
#         write_PK_Imagen['pred_dim']=ModelKeys['pred_dim']
#         write_PK_Imagen['loss_type']=ModelKeys['loss_type'] # 0 # MSE
#         write_PK_Imagen['elucidated']=ModelKeys['elucidated'] # True
#         write_PK_Imagen['padding_idx']=ModelKeys['padding_idx'] # 0 # need to check
#         # write_PK_Imagen['cond_dim']=None # use default
#         # write_PK_Imagen['text_embed_dim']=None
#         # write_PK_Imagen['input_tokens']=None
#         # write_PK_Imagen['sequence_embed']=None
#         # write_PK_Imagen['embed_dim_position']=None
#         write_PK_Imagen['max_text_len']=ModelKeys['max_text_len']
#         write_PK_Imagen['cond_images_channels']=ModelKeys['pred_dim']
#         write_PK_Imagen['max_length']=ModelKeys['max_length']
#         write_PK_Imagen['device']=ModelKeys['device']

#         # extend it to a full key
#         ModelKeys['Imagen']=UtilityPack.prepare_ModelB_keys(write_PK_Imagen)

    elif CKeys['Problem_ID']==6:
        # =====================================================
        # ForcePath --> AA sequence
        # =====================================================
        #
        # this is 1st run....
        # +++++++++++++++++++++++++++++++++++++
        # ForcPath as input seq
        ModelKeys={}
        # storage
        ModelKeys['model_dir']=PKeys['prefix']+'1_model_SS/'
        # create the folder
        print("Creating the model dir...")
        UtilityPack.create_path(ModelKeys['model_dir'])
        # secondary folders
        ModelKeys['model_dir_sample']=ModelKeys['model_dir']+'0_mid_sample/'
        UtilityPack.create_path(ModelKeys['model_dir_sample'])
        ModelKeys['model_dir_model']=ModelKeys['model_dir']+'1_store_model/'
        UtilityPack.create_path(ModelKeys['model_dir_model'])

        # for UNet
        ModelKeys['dim']=256    # dim for UNet
        ModelKeys['text_embed_dim'] = 512
        ModelKeys['embed_dim_position']=32
        ModelKeys['text_embed_dim']=ModelKeys['text_embed_dim']+ModelKeys['embed_dim_position']
        ModelKeys['cond_dim'] = 512
        ModelKeys['cond_images_channels']=DataKeys['image_channels'] # 1
        ModelKeys['max_text_len']=DataKeys['max_AA_seq_len'] # this is about text condi
        # for Imagen
        ModelKeys['pred_dim']=DataKeys['image_channels'] # 1 # for sequence, =1
        ModelKeys['diff_timesteps']=(96,)
        ModelKeys['loss_type']=0 # MSE
        ModelKeys['elucidated']=True #
        ModelKeys['padding_idx']=0
        ModelKeys['max_length']=DataKeys['max_AA_seq_len'] # max seq len
        ModelKeys['device']=device
        #
        # buckets for Model building
        ModelKeys['UNet']={}
        ModelKeys['Imagen']={}
        # ++++++++++++++++++++++++++++++++++++++++++++++++++
        write_PK_UNet=dict()
        write_PK_UNet['dim']=ModelKeys['dim'] # 256
        write_PK_UNet['text_embed_dim']=ModelKeys['text_embed_dim'] # 512+32
        write_PK_UNet['num_resnet_blocks']=1
        write_PK_UNet['cond_dim']=ModelKeys['cond_dim']  #this is where text embeddings are projected to...
        # write_PK_UNet['num_image_tokens']=None # using the default value
        # write_PK_UNet['num_time_tokens']=None  # using the default
        # write_PK_UNet['learned_sinu_pos_emb_dim']=None
        # write_PK_UNet['out_dim']=None
        write_PK_UNet['dim_mults']=(1, 2, 4, 8)

        write_PK_UNet['cond_images_channels']=ModelKeys['cond_images_channels']
        write_PK_UNet['channels']=DataKeys['image_channels'] # 1
        write_PK_UNet['channels_out']=DataKeys['image_channels'] # 1

        write_PK_UNet['attn_dim_head']=64
        write_PK_UNet['attn_heads']=8
        write_PK_UNet['ff_mult']=2.
        write_PK_UNet['lowres_cond']=False  # for cascading diffusion - https://cascaded-diffusion.github.io/

        write_PK_UNet['layer_attns']=(False, True, True, False)
        write_PK_UNet['layer_attns_depth']=1
        write_PK_UNet['layer_attns_add_text_cond']=True # whether to condition the self-attention blocks with the text embeddings, as described in Appendix D.3.1
        write_PK_UNet['attend_at_middle']=True          # whether to have a layer of attention at the bottleneck (can turn off for higher resolution in cascading DDPM, before bringing in efficient attention)
        write_PK_UNet['layer_cross_attns']=(False, True, True, False)
        write_PK_UNet['use_linear_attn']=False
        write_PK_UNet['use_linear_cross_attn']=False

        write_PK_UNet['cond_on_text']=True
        write_PK_UNet['max_text_len']=ModelKeys['max_text_len'] # need to check this one
        # ModelKeys['UNet']['init_dim']=None
        write_PK_UNet['resnet_groups']=8
        write_PK_UNet['init_conv_kernel_size']=7 # kernel size of initial conv, if not using cross embed
        write_PK_UNet['init_cross_embed']=False  #TODO - fix ouput size calcs for conv1d
        write_PK_UNet['init_cross_embed_kernel_sizes']=(3, 7, 15)
        write_PK_UNet['cross_embed_downsample']=False
        write_PK_UNet['cross_embed_downsample_kernel_sizes']=(2, 4)

        write_PK_UNet['attn_pool_text']=True
        write_PK_UNet['attn_pool_num_latents']=32 #perceiver model latents
        write_PK_UNet['dropout']=0.
        write_PK_UNet['memory_efficient']=False
        write_PK_UNet['init_conv_to_final_conv_residual']=False

        write_PK_UNet['use_global_context_attn']=True
        write_PK_UNet['scale_skip_connection']=True
        write_PK_UNet['final_resnet_block']=True
        write_PK_UNet['final_conv_kernel_size']=3

        write_PK_UNet['cosine_sim_attn']=True
        write_PK_UNet['self_cond']=False
        write_PK_UNet['combine_upsample_fmaps']=True    # combine feature maps from all upsample blocks, used in unet squared successfully
        write_PK_UNet['pixel_shuffle_upsample']=False   # may address checkboard artifacts
        # write_PK_UNet['beginning_and_final_conv_present']=None # use default
        ModelKeys['UNet']=UtilityPack.prepare_UNet_keys(write_PK_UNet)
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++
        # beyond UNet, for Whole model, all keys
        # "None" means defult value on function definition
        write_PK_Imagen=dict()

        write_PK_Imagen['timesteps']=ModelKeys['diff_timesteps']
        write_PK_Imagen['dim']=ModelKeys['dim']
        write_PK_Imagen['pred_dim']=ModelKeys['pred_dim']
        write_PK_Imagen['loss_type']=ModelKeys['loss_type'] # 0 # MSE
        write_PK_Imagen['elucidated']=ModelKeys['elucidated'] # True
        write_PK_Imagen['padding_idx']=ModelKeys['padding_idx'] # 0 # need to check
        # write_PK_Imagen['cond_dim']=None # use default
        # write_PK_Imagen['text_embed_dim']=None
        # write_PK_Imagen['input_tokens']=None
        # write_PK_Imagen['sequence_embed']=None
        # write_PK_Imagen['embed_dim_position']=None
        write_PK_Imagen['max_text_len']=ModelKeys['max_text_len']
        write_PK_Imagen['cond_images_channels']=ModelKeys['cond_images_channels']
        write_PK_Imagen['max_length']=ModelKeys['max_length']
        write_PK_Imagen['device']=ModelKeys['device']

        # extend it to a full key
        ModelKeys['Imagen']=UtilityPack.prepare_ModelB_keys(write_PK_Imagen)
        # to be defined

    elif CKeys['Problem_ID']==7:
        pass
#         # ================================================================
#         # SecStr text summary --> AA seq pLM embedding
#         # ================================================================
#         ModelKeys={}
#         # storage
#         ModelKeys['model_dir']=PKeys['prefix']+'1_model_SS/'
#         # create the folder
#         print("Creating the model dir...")
#         UtilityPack.create_path(ModelKeys['model_dir'])
#         # secondary folders
#         ModelKeys['model_dir_sample']=ModelKeys['model_dir']+'0_mid_sample/'
#         UtilityPack.create_path(ModelKeys['model_dir_sample'])
#         ModelKeys['model_dir_model']=ModelKeys['model_dir']+'1_store_model/'
#         UtilityPack.create_path(ModelKeys['model_dir_model'])

#         # for UNet
#         ModelKeys['dim']=768    # dim for UNet
#         # for debug
#         ModelKeys['dim']=256    # dim for UNet: GPU stat: 256:16093MiB;
#         #
#         ModelKeys['text_embed_dim']=512-128
#         ModelKeys['embed_dim_position']=128
#             # ! this is for UNet, not for Imagen
#         ModelKeys['text_embed_dim']=ModelKeys['text_embed_dim']+ModelKeys['embed_dim_position']
#         ModelKeys['cond_dim']=512
#         # ModelKeys['embed_dim_position']=128
#         # ModelKeys['text_embed_dim']=ModelKeys['cond_dim']-ModelKeys['embed_dim_position']
#         # !!! Need to check this one = 1 or not
#         # $: UNet use this key to decide whether cond_img is provided or not (0)
#         ModelKeys['cond_images_channels']= 0 # indicate NO cond_img  # 0 # DataKeys['image_channels']
#         # !!! Need to check this one: should it be 8 or 64?
#         ModelKeys['max_text_len']= DataKeys['max_text_len'] # DataKeys['max_AA_seq_len'] # 8 # this is about text condi

#         # for Imagen
#         ModelKeys['pred_dim']=DataKeys['image_channels'] # 1 # for sequence, =1
#         ModelKeys['diff_timesteps']=(96) # (96,)
#         ModelKeys['loss_type']=0 # MSE
#         ModelKeys['elucidated']=True #
#         ModelKeys['padding_idx']=0
#         ModelKeys['max_length']=DataKeys['max_AA_seq_len']
#         ModelKeys['device']=device
#         #
#         # baskets for Model building
#         ModelKeys['UNet']={}
#         ModelKeys['Imagen']={}
#         # ++++++++++++++++++++++++++++++++++++++
#         # prepare the Unet Key
#         write_PK_UNet=dict()
#         # used ones
#         write_PK_UNet['dim']=ModelKeys['dim']
#         write_PK_UNet['text_embed_dim']=ModelKeys['text_embed_dim']
#         write_PK_UNet['num_resnet_blocks']=1
#         write_PK_UNet['cond_dim']=ModelKeys['cond_dim'] #this is where text embeddings are projected to...
#          # write_PK_UNet['num_image_tokens']=None # using the default value
#         # write_PK_UNet['num_time_tokens']=None  # using the default
#         # write_PK_UNet['learned_sinu_pos_emb_dim']=None
#         # write_PK_UNet['out_dim']=None
#         write_PK_UNet['dim_mults']=(1, 2, 4, 8)

#         write_PK_UNet['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_UNet['channels']=DataKeys['image_channels'] # ModelKeys['pred_dim']
#         write_PK_UNet['channels_out']=DataKeys['image_channels'] # ModelKeys['pred_dim']

#         write_PK_UNet['attn_dim_head']=64
#         write_PK_UNet['attn_heads']=8
#         write_PK_UNet['ff_mult']=2.
#         write_PK_UNet['lowres_cond']=False # for cascading diffusion - https://cascaded-diffusion.github.io/


#         write_PK_UNet['layer_attns']=(False, True, True, False)
#         write_PK_UNet['layer_attns_depth']=1
#         write_PK_UNet['layer_attns_add_text_cond']=True # whether to condition the self-attention blocks with the text embeddings, as described in Appendix D.3.1
#         write_PK_UNet['attend_at_middle']=True # whether to have a layer of attention at the bottleneck (can turn off for higher resolution in cascading DDPM, before bringing in efficient attention)
#         write_PK_UNet['layer_cross_attns']=(False, True, True, False)
#         write_PK_UNet['use_linear_attn']=False
#         write_PK_UNet['use_linear_cross_attn']=False

#         write_PK_UNet['cond_on_text'] = True
#             # !!! Need to check this: 63, Imagen used a different one
#             # !!! here try the new one which seems to be correct
#         write_PK_UNet['max_text_len'] = ModelKeys['max_text_len'] # DataKeys['max_AA_seq_len'] # ModelKeys['max_text_len']
#         # write_PK_UNet['init_dim'] = None
#         write_PK_UNet['resnet_groups'] = 8
#         write_PK_UNet['init_conv_kernel_size'] =7     # kernel size of initial conv, if not using cross embed
#         write_PK_UNet['init_cross_embed'] = False  #TODO - fix ouput size calcs for conv1d
#         write_PK_UNet['init_cross_embed_kernel_sizes'] = (3, 7, 15)
#         write_PK_UNet['cross_embed_downsample'] = False
#         write_PK_UNet['cross_embed_downsample_kernel_sizes'] = (2, 4)

#         write_PK_UNet['attn_pool_text'] = True
#         write_PK_UNet['attn_pool_num_latents'] = 32 #32,   #perceiver model latents
#         write_PK_UNet['dropout'] = 0.
#         write_PK_UNet['memory_efficient'] = False
#         write_PK_UNet['init_conv_to_final_conv_residual'] = False

#         write_PK_UNet['use_global_context_attn'] = True
#         write_PK_UNet['scale_skip_connection'] = True
#         write_PK_UNet['final_resnet_block'] = True
#         write_PK_UNet['final_conv_kernel_size'] = 3

#         write_PK_UNet['cosine_sim_attn'] = True
#         write_PK_UNet['self_cond'] = False
#         write_PK_UNet['combine_upsample_fmaps'] = True      # combine feature maps from all upsample blocks, used in unet squared successfully
#         write_PK_UNet['pixel_shuffle_upsample'] = False     # may address checkboard artifacts
#         # write_PK_UNet['beginning_and_final_conv_present']=None # use default
#         #
#         ModelKeys['UNet']=UtilityPack.prepare_UNet_keys(write_PK_UNet)
#         # ++++++++++++++++++++++++++++++++++++++
#         # beyond UNet, for Whole model, all keys
#         # "None" means defult value on function definition
#         write_PK_Imagen=dict()

#         # used ones
#         write_PK_Imagen['timesteps']=ModelKeys['diff_timesteps']
#         write_PK_Imagen['dim']=ModelKeys['dim']
#         write_PK_Imagen['pred_dim']=ModelKeys['pred_dim']
#         write_PK_Imagen['loss_type']=ModelKeys['loss_type'] # 0 # MSE
#         write_PK_Imagen['elucidated']=ModelKeys['elucidated'] # True
#         write_PK_Imagen['padding_idx']=ModelKeys['padding_idx'] # 0 # need to check
#         write_PK_Imagen['cond_dim']=ModelKeys['cond_dim'] # use default
#             # can use default, 512; or like the below (check the code)
#         write_PK_Imagen['text_embed_dim']=ModelKeys['text_embed_dim']-ModelKeys['embed_dim_position']
#         # write_PK_Imagen['input_tokens']=None
#         # write_PK_Imagen['sequence_embed']=None
#         write_PK_Imagen['embed_dim_position']=ModelKeys['embed_dim_position']
#         write_PK_Imagen['max_text_len']=ModelKeys['max_text_len']
#         write_PK_Imagen['cond_images_channels']=ModelKeys['cond_images_channels']
#         write_PK_Imagen['max_length']=ModelKeys['max_length']
#         write_PK_Imagen['device']=ModelKeys['device']

#         # extend it to a full key
#         ModelKeys['Imagen']=UtilityPack.prepare_ModelB_keys(write_PK_Imagen)

    elif CKeys['Problem_ID']==8:
        # ================================================================
        # FmaxEne text summary --> AA seq pLM embedding
        # ================================================================
        ModelKeys={}
        # storage
        ModelKeys['model_dir']=PKeys['prefix']+'1_model_SS/'
        # create the folder
        print("Creating the model dir...")
        UtilityPack.create_path(ModelKeys['model_dir'])
        # secondary folders
        ModelKeys['model_dir_sample']=ModelKeys['model_dir']+'0_mid_sample/'
        UtilityPack.create_path(ModelKeys['model_dir_sample'])
        ModelKeys['model_dir_model']=ModelKeys['model_dir']+'1_store_model/'
        UtilityPack.create_path(ModelKeys['model_dir_model'])

        # for UNet
        ModelKeys['dim']=768    # dim for UNet
        # for debug
        ModelKeys['dim']=256    # dim for UNet: GPU stat: 256:16093MiB;
        #
        ModelKeys['text_embed_dim']=512-128
        ModelKeys['embed_dim_position']=128
            # ! this is for UNet, not for Imagen
        ModelKeys['text_embed_dim']=ModelKeys['text_embed_dim']+ModelKeys['embed_dim_position']
        ModelKeys['cond_dim']=512
        # ModelKeys['embed_dim_position']=128
        # ModelKeys['text_embed_dim']=ModelKeys['cond_dim']-ModelKeys['embed_dim_position']
        # !!! Need to check this one = 1 or not
        # $: UNet use this key to decide whether cond_img is provided or not (0)
        ModelKeys['cond_images_channels']=0 # indicate no cond_img is used # 1
        # !!! Need to check this one: should it be 8 or 64?
        ModelKeys['max_text_len']= DataKeys['max_text_len'] # DataKeys['max_AA_seq_len'] # 8 # this is about text condi

        # for Imagen
        ModelKeys['pred_dim']=DataKeys['image_channels'] # 1 # for sequence, =1
        ModelKeys['diff_timesteps']=(96) # (96,)
        ModelKeys['loss_type']=0 # MSE
        ModelKeys['elucidated']=True #
        ModelKeys['padding_idx']=0
        ModelKeys['max_length']=DataKeys['max_AA_seq_len']
        ModelKeys['device']=device
        #
        # baskets for Model building
        ModelKeys['UNet']={}
        ModelKeys['Imagen']={}
        # ++++++++++++++++++++++++++++++++++++++
        # prepare the Unet Key
        write_PK_UNet=dict()
        # used ones
        write_PK_UNet['dim']=ModelKeys['dim']
        write_PK_UNet['text_embed_dim']=ModelKeys['text_embed_dim']
        write_PK_UNet['num_resnet_blocks']=1
        write_PK_UNet['cond_dim']=ModelKeys['cond_dim'] #this is where text embeddings are projected to...
         # write_PK_UNet['num_image_tokens']=None # using the default value
        # write_PK_UNet['num_time_tokens']=None  # using the default
        # write_PK_UNet['learned_sinu_pos_emb_dim']=None
        # write_PK_UNet['out_dim']=None
        write_PK_UNet['dim_mults']=(1, 2, 4, 8)

        write_PK_UNet['cond_images_channels']=ModelKeys['cond_images_channels']
        write_PK_UNet['channels']=DataKeys['image_channels']  # ModelKeys['pred_dim']
        write_PK_UNet['channels_out']=DataKeys['image_channels'] # ModelKeys['pred_dim']

        write_PK_UNet['attn_dim_head']=64
        write_PK_UNet['attn_heads']=8
        write_PK_UNet['ff_mult']=2.
        write_PK_UNet['lowres_cond']=False # for cascading diffusion - https://cascaded-diffusion.github.io/


        write_PK_UNet['layer_attns']=(False, True, True, False)
        write_PK_UNet['layer_attns_depth']=1
        write_PK_UNet['layer_attns_add_text_cond']=True # whether to condition the self-attention blocks with the text embeddings, as described in Appendix D.3.1
        write_PK_UNet['attend_at_middle']=True # whether to have a layer of attention at the bottleneck (can turn off for higher resolution in cascading DDPM, before bringing in efficient attention)
        write_PK_UNet['layer_cross_attns']=(False, True, True, False)
        write_PK_UNet['use_linear_attn']=False
        write_PK_UNet['use_linear_cross_attn']=False

        write_PK_UNet['cond_on_text'] = True
            # !!! Need to check this: 63, Imagen used a different one
        write_PK_UNet['max_text_len'] = DataKeys['max_text_len'] # DataKeys['max_AA_seq_len'] # ModelKeys['max_text_len']
        # write_PK_UNet['init_dim'] = None
        write_PK_UNet['resnet_groups'] = 8
        write_PK_UNet['init_conv_kernel_size'] =7     # kernel size of initial conv, if not using cross embed
        write_PK_UNet['init_cross_embed'] = False  #TODO - fix ouput size calcs for conv1d
        write_PK_UNet['init_cross_embed_kernel_sizes'] = (3, 7, 15)
        write_PK_UNet['cross_embed_downsample'] = False
        write_PK_UNet['cross_embed_downsample_kernel_sizes'] = (2, 4)

        write_PK_UNet['attn_pool_text'] = True
        write_PK_UNet['attn_pool_num_latents'] = 32 #32,   #perceiver model latents
        write_PK_UNet['dropout'] = 0.
        write_PK_UNet['memory_efficient'] = False
        write_PK_UNet['init_conv_to_final_conv_residual'] = False

        write_PK_UNet['use_global_context_attn'] = True
        write_PK_UNet['scale_skip_connection'] = True
        write_PK_UNet['final_resnet_block'] = True
        write_PK_UNet['final_conv_kernel_size'] = 3

        write_PK_UNet['cosine_sim_attn'] = True
        write_PK_UNet['self_cond'] = False
        write_PK_UNet['combine_upsample_fmaps'] = True      # combine feature maps from all upsample blocks, used in unet squared successfully
        write_PK_UNet['pixel_shuffle_upsample'] = False     # may address checkboard artifacts
        # write_PK_UNet['beginning_and_final_conv_present']=None # use default
        #
        ModelKeys['UNet']=UtilityPack.prepare_UNet_keys(write_PK_UNet)
        # ++++++++++++++++++++++++++++++++++++++
        # beyond UNet, for Whole model, all keys
        # "None" means defult value on function definition
        write_PK_Imagen=dict()

        # used ones
        write_PK_Imagen['timesteps']=ModelKeys['diff_timesteps']
        write_PK_Imagen['dim']=ModelKeys['dim']
        write_PK_Imagen['pred_dim']=ModelKeys['pred_dim']
        write_PK_Imagen['loss_type']=ModelKeys['loss_type'] # 0 # MSE
        write_PK_Imagen['elucidated']=ModelKeys['elucidated'] # True
        write_PK_Imagen['padding_idx']=ModelKeys['padding_idx'] # 0 # need to check
        write_PK_Imagen['cond_dim']=ModelKeys['cond_dim'] # use default
            # can use default, 512; or like the below (check the code)
        write_PK_Imagen['text_embed_dim']=ModelKeys['text_embed_dim']-ModelKeys['embed_dim_position']
        # write_PK_Imagen['input_tokens']=None
        # write_PK_Imagen['sequence_embed']=None
        write_PK_Imagen['embed_dim_position']=ModelKeys['embed_dim_position']
        write_PK_Imagen['max_text_len']=ModelKeys['max_text_len']
        write_PK_Imagen['cond_images_channels']=ModelKeys['cond_images_channels']
        write_PK_Imagen['max_length']=ModelKeys['max_length']
        write_PK_Imagen['device']=ModelKeys['device']

        # extend it to a full key
        ModelKeys['Imagen']=UtilityPack.prepare_ModelB_keys(write_PK_Imagen)

    elif CKeys['Problem_ID']==11:
        # =====================================================
        # ForcePath --> AA sequence
        # =====================================================
        #
        # this is 1st run....
        # +++++++++++++++++++++++++++++++++++++
        # ForcPath as input seq
        ModelKeys={}
        # storage
        ModelKeys['model_dir']=PKeys['prefix']+'1_model_SS/'
        # create the folder
        print("Creating the model dir...")
        UtilityPack.create_path(ModelKeys['model_dir'])
        # secondary folders
        ModelKeys['model_dir_sample']=ModelKeys['model_dir']+'0_mid_sample/'
        UtilityPack.create_path(ModelKeys['model_dir_sample'])
        ModelKeys['model_dir_model']=ModelKeys['model_dir']+'1_store_model/'
        UtilityPack.create_path(ModelKeys['model_dir_model'])

        # for UNet
        ModelKeys['dim']=256    # dim for UNet
        ModelKeys['text_embed_dim'] = 512
        ModelKeys['embed_dim_position']=32
        ModelKeys['text_embed_dim']=ModelKeys['text_embed_dim']+ModelKeys['embed_dim_position']
        ModelKeys['cond_dim'] = 512
        ModelKeys['cond_images_channels']=DataKeys['image_channels'] # 1
        ModelKeys['max_text_len']=DataKeys['max_AA_seq_len'] # this is about text condi
        # for Imagen
        ModelKeys['pred_dim']=DataKeys['image_channels'] # 1 # for sequence, =1
        ModelKeys['diff_timesteps']=(96,)
        ModelKeys['loss_type']=0 # MSE
        ModelKeys['elucidated']=True #
        ModelKeys['padding_idx']=0
        ModelKeys['max_length']=DataKeys['max_AA_seq_len'] # max seq len
        ModelKeys['device']=device
        #
        # buckets for Model building
        ModelKeys['UNet']={}
        ModelKeys['Imagen']={}
        # ++++++++++++++++++++++++++++++++++++++++++++++++++
        write_PK_UNet=dict()
        write_PK_UNet['dim']=ModelKeys['dim'] # 256
        write_PK_UNet['text_embed_dim']=ModelKeys['text_embed_dim'] # 512+32
        write_PK_UNet['num_resnet_blocks']=1
        write_PK_UNet['cond_dim']=ModelKeys['cond_dim']  #this is where text embeddings are projected to...
        # write_PK_UNet['num_image_tokens']=None # using the default value
        # write_PK_UNet['num_time_tokens']=None  # using the default
        # write_PK_UNet['learned_sinu_pos_emb_dim']=None
        # write_PK_UNet['out_dim']=None
        write_PK_UNet['dim_mults']=(1, 2, 4, 8)

        write_PK_UNet['cond_images_channels']=ModelKeys['cond_images_channels']
        write_PK_UNet['channels']=DataKeys['image_channels'] # 1
        write_PK_UNet['channels_out']=DataKeys['image_channels'] # 1

        write_PK_UNet['attn_dim_head']=64
        write_PK_UNet['attn_heads']=8
        write_PK_UNet['ff_mult']=2.
        write_PK_UNet['lowres_cond']=False  # for cascading diffusion - https://cascaded-diffusion.github.io/

        write_PK_UNet['layer_attns']=(False, True, True, False)
        write_PK_UNet['layer_attns_depth']=1
        write_PK_UNet['layer_attns_add_text_cond']=True # whether to condition the self-attention blocks with the text embeddings, as described in Appendix D.3.1
        write_PK_UNet['attend_at_middle']=True          # whether to have a layer of attention at the bottleneck (can turn off for higher resolution in cascading DDPM, before bringing in efficient attention)
        write_PK_UNet['layer_cross_attns']=(False, True, True, False)
        write_PK_UNet['use_linear_attn']=False
        write_PK_UNet['use_linear_cross_attn']=False

        write_PK_UNet['cond_on_text']=True
        write_PK_UNet['max_text_len']=ModelKeys['max_text_len'] # need to check this one
        # ModelKeys['UNet']['init_dim']=None
        write_PK_UNet['resnet_groups']=8
        write_PK_UNet['init_conv_kernel_size']=7 # kernel size of initial conv, if not using cross embed
        write_PK_UNet['init_cross_embed']=False  #TODO - fix ouput size calcs for conv1d
        write_PK_UNet['init_cross_embed_kernel_sizes']=(3, 7, 15)
        write_PK_UNet['cross_embed_downsample']=False
        write_PK_UNet['cross_embed_downsample_kernel_sizes']=(2, 4)

        write_PK_UNet['attn_pool_text']=True
        write_PK_UNet['attn_pool_num_latents']=32 #perceiver model latents
        write_PK_UNet['dropout']=0.
        write_PK_UNet['memory_efficient']=False
        write_PK_UNet['init_conv_to_final_conv_residual']=False

        write_PK_UNet['use_global_context_attn']=True
        write_PK_UNet['scale_skip_connection']=True
        write_PK_UNet['final_resnet_block']=True
        write_PK_UNet['final_conv_kernel_size']=3

        write_PK_UNet['cosine_sim_attn']=True
        write_PK_UNet['self_cond']=False
        write_PK_UNet['combine_upsample_fmaps']=True    # combine feature maps from all upsample blocks, used in unet squared successfully
        write_PK_UNet['pixel_shuffle_upsample']=False   # may address checkboard artifacts
        # write_PK_UNet['beginning_and_final_conv_present']=None # use default
        ModelKeys['UNet']=UtilityPack.prepare_UNet_keys(write_PK_UNet)
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++
        # beyond UNet, for Whole model, all keys
        # "None" means defult value on function definition
        write_PK_Imagen=dict()

        write_PK_Imagen['timesteps']=ModelKeys['diff_timesteps']
        write_PK_Imagen['dim']=ModelKeys['dim']
        write_PK_Imagen['pred_dim']=ModelKeys['pred_dim']
        write_PK_Imagen['loss_type']=ModelKeys['loss_type'] # 0 # MSE
        write_PK_Imagen['elucidated']=ModelKeys['elucidated'] # True
        write_PK_Imagen['padding_idx']=ModelKeys['padding_idx'] # 0 # need to check
        # write_PK_Imagen['cond_dim']=None # use default
        # write_PK_Imagen['text_embed_dim']=None
        # write_PK_Imagen['input_tokens']=None
        # write_PK_Imagen['sequence_embed']=None
        # write_PK_Imagen['embed_dim_position']=None
        write_PK_Imagen['max_text_len']=ModelKeys['max_text_len']
        write_PK_Imagen['cond_images_channels']=ModelKeys['cond_images_channels']
        write_PK_Imagen['max_length']=ModelKeys['max_length']
        write_PK_Imagen['device']=ModelKeys['device']

        # extend it to a full key
        ModelKeys['Imagen']=UtilityPack.prepare_ModelB_keys(write_PK_Imagen)
        # to be defined


    # --
    print("==================================================")
    print("store the MODEL key for the next-time usage")
    print("==================================================")
    model_pack = {}
    model_pack['ModelKeys']=ModelKeys
    with open(PKeys['pk_model_pack'], 'wb') as handle:
        pickle.dump(model_pack, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print("Done.")


else: # both for training and testing
    #
    print("==================================================")
    print("load in the MODEL key from the previous storage")
    print("==================================================")
    #
    # this is not the first run
    print('This is not the first run')
    print('Load back in the model packages...')
    with open(PKeys['pk_model_pack'], 'rb') as handle:
        model_pack = pickle.load(handle)
    # deliver the results
    ModelKeys=model_pack['ModelKeys']
    print('Done.')



load in the MODEL key from the previous storage
This is not the first run
Load back in the model packages...
Done.


In [63]:
# print(DataKeys)
# check a few key words
# print (DataKeys['max_text_len'])
# print(DataKeys['max_text_len'])
print(ModelKeys['max_text_len'])
print(DataKeys['image_channels'])
print(ModelKeys['cond_images_channels'])
print(ModelKeys['pred_dim'])

128
33
33
33


In [65]:
print(json.dumps(ModelKeys['UNet'], indent=4))
# print(json.dumps(ModelKeys['Imagen'], indent=4))
print(ModelKeys['Imagen'])

{
    "dim": 256,
    "text_embed_dim": 544,
    "num_resnet_blocks": 1,
    "cond_dim": 512,
    "num_image_tokens": null,
    "num_time_tokens": null,
    "learned_sinu_pos_emb_dim": null,
    "out_dim": null,
    "dim_mults": [
        1,
        2,
        4,
        8
    ],
    "cond_images_channels": 33,
    "channels": 33,
    "channels_out": 33,
    "attn_dim_head": 64,
    "attn_heads": 8,
    "ff_mult": 2.0,
    "lowres_cond": false,
    "layer_attns": [
        false,
        true,
        true,
        false
    ],
    "layer_attns_depth": 1,
    "layer_attns_add_text_cond": true,
    "attend_at_middle": true,
    "layer_cross_attns": [
        false,
        true,
        true,
        false
    ],
    "use_linear_attn": false,
    "use_linear_cross_attn": false,
    "cond_on_text": true,
    "max_text_len": 128,
    "init_dim": null,
    "resnet_groups": 8,
    "init_conv_kernel_size": 7,
    "init_cross_embed": false,
    "init_cross_embed_kernel_sizes": [
        3,
  

In [66]:

# select some key values
print(ModelKeys['UNet']['channels'])
print(ModelKeys['UNet']['channels_out'])
print(ModelKeys['UNet']['cond_images_channels'])
print(ModelKeys['Imagen']['pred_dim'])
print(ModelKeys['Imagen']['cond_images_channels'])
# how large the model will be
print()
print(ModelKeys['UNet']['dim'])
print(ModelKeys['Imagen']['dim'])

33
33
33
33
33

256
256


In [67]:
# =====================================
# setup the model
# =====================================
if CKeys['Problem_ID']==1 or CKeys['Problem_ID']==2:
    pass
#     # =================================================
#     # Resideual level tasks: Model B
#     # =================================================

#     # working code: assemble the model archi
#     print('--------------------------------------------')
#     print('On OneD_Unet...')
#     print('--------------------------------------------')
#     working_unet = ModelPack.OneD_Unet(
#         CKeys=CKeys,
#         PKeys=ModelKeys['UNet'],
#     ).to(device)

#     print("Working unet part model: ")
#     UtilityPack.params ( working_unet)

#     print('--------------------------------------------')
#     print('On whole model...')
#     print('--------------------------------------------')
#     # on the model part
#     working_model = ModelPack.ProteinDesigner_B(
#         working_unet,
#         CKeys=CKeys,
#         PKeys=ModelKeys['Imagen'],
#     ). to(device)

#     # measure
#     print ("Total working model: ")
#     UtilityPack.params ( working_model)
#     print ("Recasted unet inside the tot one only: ")
#     UtilityPack.params ( working_model.imagen.unets[0])

#     print('--------------------------------------------')
#     print('On trainer...')
#     print('--------------------------------------------')
#     working_trainer = TrainerPack.ImagenTrainer(
#         working_model,
#         CKeys=CKeys
#     )

elif CKeys['Problem_ID']==3 or CKeys['Problem_ID']==4:
    pass
#     # =================================================
#     # Sequence level tasks: Model A
#     # =================================================

#     # working code: assemble the model archi
#     print('--------------------------------------------')
#     print('On OneD_Unet...')
#     print('--------------------------------------------')
#     working_unet = ModelPack.OneD_Unet(
#         CKeys=CKeys,
#         PKeys=ModelKeys['UNet'],
#     ).to(device)

#     print("Working unet part model: ")
#     UtilityPack.params ( working_unet)

#     print('--------------------------------------------')
#     print('On whole model...')
#     print('--------------------------------------------')
#     # on the model part
#     working_model = ModelPack.ProteinDesigner_A_II(
#         working_unet,
#         CKeys=CKeys,
#         PKeys=ModelKeys['Imagen'],
#     ). to(device)

#     # measure
#     print ("Total working model: ")
#     UtilityPack.params ( working_model)
#     print ("Recasted unet inside the tot one only: ")
#     UtilityPack.params ( working_model.imagen.unets[0])

#     print('--------------------------------------------')
#     print('On trainer...')
#     print('--------------------------------------------')
#     working_trainer = TrainerPack.ImagenTrainer(
#         working_model,
#         CKeys=CKeys
#     )

elif CKeys['Problem_ID']==5 or CKeys['Problem_ID']==6 \
or   CKeys['Problem_ID']==11:
    # =================================================
    # Resideual level tasks: Model B
    # =================================================

    # working code: assemble the model archi
    print('--------------------------------------------')
    print('On OneD_Unet...')
    print('--------------------------------------------')
    working_unet = ModelPack.OneD_Unet(
        CKeys=CKeys,
        PKeys=ModelKeys['UNet'],
    ).to(device)

    print("Working unet part model: ")
    UtilityPack.params ( working_unet)

    print('--------------------------------------------')
    print('On whole model...')
    print('--------------------------------------------')
    # on the model part
    working_model = ModelPack.ProteinDesigner_B(
        working_unet,
        CKeys=CKeys,
        PKeys=ModelKeys['Imagen'],
    ). to(device)

    # measure
    print ("Total working model: ")
    UtilityPack.params ( working_model)
    print ("Recasted unet inside the tot one only: ")
    UtilityPack.params ( working_model.imagen.unets[0])

    print('--------------------------------------------')
    print('On trainer...')
    print('--------------------------------------------')
    working_trainer = TrainerPack.ImagenTrainer(
        working_model,
        CKeys=CKeys
    )

elif CKeys['Problem_ID']==7 or CKeys['Problem_ID']==8:
    # =================================================
    # Sequence level tasks: Model A
    # =================================================

    # working code: assemble the model archi
    print('--------------------------------------------')
    print('On OneD_Unet...')
    print('--------------------------------------------')
    working_unet = ModelPack.OneD_Unet(
        CKeys=CKeys,
        PKeys=ModelKeys['UNet'],
    ).to(device)

    print("Working unet part model: ")
    UtilityPack.params ( working_unet)

    print('--------------------------------------------')
    print('On whole model...')
    print('--------------------------------------------')
    # on the model part
    working_model = ModelPack.ProteinDesigner_A_II(
        working_unet,
        CKeys=CKeys,
        PKeys=ModelKeys['Imagen'],
    ). to(device)

    # measure
    print ("Total working model: ")
    UtilityPack.params ( working_model)
    print ("Recasted unet inside the tot one only: ")
    UtilityPack.params ( working_model.imagen.unets[0])

    print('--------------------------------------------')
    print('On trainer...')
    print('--------------------------------------------')
    working_trainer = TrainerPack.ImagenTrainer(
        working_model,
        CKeys=CKeys
    )



--------------------------------------------
On OneD_Unet...
--------------------------------------------
256 33
Working unet part model: 
Total parameters:  264376242  trainable parameters:  264376242
--------------------------------------------
On whole model...
--------------------------------------------
Model B: Generative protein diffusion model, residue-based
Using condition as the initial sequence
Use conditioning image during training....
Loss type:  0
Channels in=33, channels out=33
Test on cast_model_parameters...
False
False
None
33
33
256 33
cpu
Total working model: 
Total parameters:  262527570  trainable parameters:  262527570
Recasted unet inside the tot one only: 
Total parameters:  262521266  trainable parameters:  262521266
--------------------------------------------
On trainer...
--------------------------------------------


In [68]:

del working_unet
torch.cuda.empty_cache()

### 1.4. Training part

In [69]:

if CKeys['Working_Mode']==1:
    print("Training mode...")

    TrainKeys={}
    if CKeys['Debug']==1:
        TrainKeys['epochs']=4
        TrainKeys['print_loss_every_this_epochs']=1
        TrainKeys['sample_every_this_epochs']=1
        TrainKeys['save_model_every_this_epochs']=2
    else:
        TrainKeys['epochs']=CKeys['epochs'] # 200
        TrainKeys['print_loss_every_this_epochs']=CKeys['print_loss_every_this_epochs'] # 5
        TrainKeys['sample_every_this_epochs']=CKeys['sample_every_this_epochs'] # 10
        TrainKeys['save_model_every_this_epochs']=CKeys['save_model_every_this_epochs'] # 20

In [70]:

# one mini batch
if CKeys['Debug']==1 and CKeys['Debug_DataPack']==1:
    print("in a mini-batch:")
    print("input.dim:  ", this_item[0].shape)
    # print(this_item[0][0,:]*torch.FloatTensor(DataKeys['Xnormfac']))
    print("output.dim: ", this_item[1].shape)
    print(DataKeys['Xnormfac'])

In [71]:

if CKeys['Working_Mode']==1:
    # ==============================================================
    # copy from the above when debug is done:
    # ==============================================================
    # a few files for restarts:
    TRAINING_HIST_FILE   = ModelKeys['model_dir']+'Training_Hist.csv'
    TRAINING_HIST_FILE_FULL = ModelKeys['model_dir']+'Training_Hist_Full.csv'
    TRAINING_BREAK_POINT = ModelKeys['model_dir']+'Training_Info.txt'

    if CKeys['Problem_ID']==1:
        pass
        # # de novo test
        # test_condition_list = [
        #     ['~~~HHHHHHHHHHHHHHH~~'],
        #     ['~~EEESSTTS~SEEEEEEEEE~SBS~EEEEEE~~'],
        # ]

    elif CKeys['Problem_ID']==2:
        pass

        # test_condition_list = [
        #     np.expand_dims(np.array(DataSetPack.pad_a_np_arr(protein_df['sample_NormPullGap_data'][pick_id_0],0,64))*DataKeys['Xnormfac']*0.33, axis=0),
        #     np.expand_dims(np.array(DataSetPack.pad_a_np_arr(protein_df['sample_NormPullGap_data'][pick_id_0],0,64))*DataKeys['Xnormfac']*0.66, axis=0),
        # ]

    # de novo test
    elif CKeys['Problem_ID']==3:
        pass

        # test_condition_list = [
        #     [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        #     [0, 0.7, 0.07, 0.1, 0.01, 0.02, 0.01, 0.11],
        # ]

    elif CKeys['Problem_ID']==4:
        pass

        # # here we used normalized (nFmax, nToughness)
        # # the real values will be (nFmax, nToughness)*Xnormfac
        # test_condition_list = [
        #     [0.5, 0.5],
        #     [0.2, 0.8],
        # ]

    # # #
    # # test_condition_list = [
    # #     np.expand_dims(np.array(this_item[0][10,:]*DataKeys['Xnormfac']), axis=0),
    # #     np.expand_dims(np.array(this_item[0][20,:]*DataKeys['Xnormfac']), axis=0),
    # # ]
    # # protein_df['sample_NormPullGap_data'][pick_id_0]
    # # DataSetPack.pad_a_np_arr(protein_df['sample_NormPullGap_data'][pick_id_0],0,64)
    # # #
    # test_condition_list = [
    #     np.expand_dims(np.array(DataSetPack.pad_a_np_arr(protein_df['sample_NormPullGap_data'][pick_id_0],0,64))*DataKeys['Xnormfac']*0.33, axis=0),
    #     np.expand_dims(np.array(DataSetPack.pad_a_np_arr(protein_df['sample_NormPullGap_data'][pick_id_0],0,64))*DataKeys['Xnormfac']*0.66, axis=0),
    # ]
    elif CKeys['Problem_ID']==5:
        pass

        # # de novo test
        # test_condition_list = [
        #     ['~~~HHHHHHHHHHHHHHH~~'],
        #     ['~~EEESSTTS~SEEEEEEEEE~SBS~EEEEEE~~'],
        # ]

    #
    elif CKeys['Problem_ID']==6 or CKeys['Problem_ID']==11:
        # for ESM model: 0+content+00
        # test_condition_list = [
        #     np.expand_dims(np.array([0]+DataSetPack.pad_a_np_arr(protein_df['sample_NormPullGap_data'][pick_id_0],0,64-1))*DataKeys['Xnormfac']*0.33, axis=0),
        #     np.expand_dims(np.array([0]+DataSetPack.pad_a_np_arr(protein_df['sample_NormPullGap_data'][pick_id_0],0,64-1))*DataKeys['Xnormfac']*0.66, axis=0),
        # ]
        #
        # test_condition_list = [
        #     np.insert(np.array(DataSetPack.pad_a_np_arr(protein_df['sample_NormPullGap_data'][pick_id_0],0,64-1))*DataKeys['Xnormfac']*0.33, 0, 0.),
        #     np.insert(np.array(DataSetPack.pad_a_np_arr(protein_df['sample_NormPullGap_data'][pick_id_0],0,64-1))*DataKeys['Xnormfac']*0.66, 0, 0.),
        # ]
        #
        test_0 = DataSetPack.pad_a_np_arr(
            protein_df['sample_FORCE_data'][pick_id_0],
            0.,
            DataKeys['max_AA_seq_len']
        )
        test_1 = DataSetPack.pad_a_np_arr_esm(
            protein_df['sample_FORCE_data'][pick_id_1],
            0.,
            DataKeys['max_AA_seq_len']
        )
        # test_condition_list = [
        #     test_0*0.33,
        #     test_0*0.66,
        #     test_1*0.33,
        #     test_1*0.66,
        # ]
        test_condition_list = [
            test_0*0.66,
            test_1*1.66,
        ]


    #
    # de novo test
    elif CKeys['Problem_ID']==7:
        pass

        # test_condition_list = [
        #     [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        #     [0, 0.7, 0.07, 0.1, 0.01, 0.02, 0.01, 0.11],
        # ]

    elif CKeys['Problem_ID']==8:

        test_condition_list = [
            [0.2, 0.8]*DataKeys['Xnormfac'],
            [0.8, 0.2]*DataKeys['Xnormfac'],
        ]


    else:
        print("No de novo condition is found for the problem...")




In [74]:
print('Check tokenizer_X:', tokenizer_X)
print('tokenizer_y:', tokenizer_y)

Check tokenizer_X: None
tokenizer_y: None


In [75]:
# skip training part

### 1.5. Testing loop

In [77]:
if CKeys['Working_Mode']==2:
    print("This is testing model...")
    # setup test key
    TestKeys={}
    TestKeys['Sample_On_TestSet']=False # True # True # False
    TestKeys['Sample_On_DeNovo']=True
    # create a test dir
    # TestKeys['test_dir']=PKeys['prefix']+'2_test/'
    # TestKeys['test_dir']=PKeys['prefix']+'3_test_common_denovo/'
    # add for colab
    TestKeys['test_dir']=PKeys['prefix']+'4_test_individual_cases/'
    UtilityPack.create_path(TestKeys['test_dir'])

This is testing model...
Creating the given path...
Done.


In [76]:
print(PKeys['prefix'])

/content/working_results/


In [82]:


if CKeys['Working_Mode']==2:
    # skip for colab inferring
    pass
    # # looking back at the training
    # print("Check the training history...")
    # #
    # # training history
    # TRAINING_HIST_FILE   = ModelKeys['model_dir']+'Training_Hist.csv'
    # TRAINING_HIST_FILE_FULL = ModelKeys['model_dir']+'Training_Hist_Full.csv'
    # TRAINING_BREAK_POINT = ModelKeys['model_dir']+'Training_Info.txt'

    # # pick up the leftover info
    # # train_rec = pd.read_csv(TRAINING_HIST_FILE)
    # train_rec = pd.read_csv(TRAINING_HIST_FILE_FULL)
    # #
    # id_best_loss = np.argmin(train_rec['norm_loss'])
    # best_epoch = train_rec['epoch'][id_best_loss]
    # # print(best_epoch)
    # print(f"Best epoch: {best_epoch}; LOSS: {train_rec['norm_loss'][id_best_loss]}", )

    # fig = plt.figure()
    # plt.plot(
    #     train_rec['epoch'],
    #     train_rec['norm_loss'],
    #     label='Loss',
    # )
    # plt.legend()
    # outname=TestKeys['test_dir']+"0_Training_Hist.jpg"
    # if CKeys['SlientRun']==1:
    #     plt.savefig(outname, dpi=200)
    # else:
    #     pass
    # plt.show()
    # plt.close(fig)

    # #
    # # pick up the breaking point of last time
    # breaking_rec = pd.read_csv(TRAINING_BREAK_POINT)
    # last_epoch = breaking_rec['epoch'][0]
    # last_step  = breaking_rec['steps'][0]
    # # print("Last epoch: ", breaking_rec['epoch'][0])
    # print(f"Last epoch: {last_epoch}; LOSS: {breaking_rec['norm_loss'][0]}", )
    #
    # last_epoch = 4000
    # last_step = 92000



In [ ]:
if CKeys['Working_Mode']==2:
    print("Load back the model at the LAST epoch...")
    # load back the LAST model:
    # model_last_fname=f"{ModelKeys['model_dir_model']}trainer_save-model-epoch_{last_epoch}.pt"
    model_last_fname=f"{this_working_path}1_model_SS/trainer_save-model_pLDM.pt"
    working_trainer.load(model_last_fname)
    # TBA: load back the Best model AVAILABLE

In [81]:
print(ModelKeys['model_dir_model'])
# print(last_epoch)

/home/gridsan/bni/16_WG_git_sort_pdb_database_0/11_pLMProb_Diff_SMD_ModelB_embed_640/0_Training/1_model_SS/1_store_model/


In [37]:
PKeys['prefix']=this_working_path
PKeys['pk_data_pack']=this_working_path+'data_pack.pickle'
PKeys['pk_model_pack']=this_working_path+'model_pack.pickle'

{'prefix': '/home/gridsan/bni/16_WG_git_sort_pdb_database_0/11_pLMProb_Diff_SMD_ModelB_embed_640/0_Training/',
 'pk_data_pack': '/home/gridsan/bni/16_WG_git_sort_pdb_database_0/11_pLMProb_Diff_SMD_ModelB_embed_640/0_Training/data_pack.pickle',
 'pk_model_pack': '/home/gridsan/bni/16_WG_git_sort_pdb_database_0/11_pLMProb_Diff_SMD_ModelB_embed_640/0_Training/model_pack.pickle'}

In [31]:
print(DataKeys['data_dir'])

/home/gridsan/bni/16_WG_git_sort_pdb_database_0/11_pLMProb_Diff_SMD_ModelB_embed_640/0_Training/0_dataprocess_MD/


In [ ]:
print(PKeys['pk_data_pack'])

/content/Trained_model/data_pack.pickle


In [ ]:
print(PKeys['pk_data_pack'])


/content/Trained_model/data_pack.pickle


In [ ]:
import ModelPack

In [ ]:
print('test')

test
